In [106]:
-- load styles & scripts to pimp the notebook.
import qualified IHaskell.Display as ID
ID.Display [
    ID.html "<link href=\"https://unpkg.com/tailwindcss@^1.0/dist/tailwind.min.css\" rel=\"stylesheet\">"
    , ID.html "<link href=\"http://chodov:9000/xstatic/css/pimping.css\" rel=\"stylesheet\">"
    , ID.html "<script src=\"http://chodov:9000/xstatic/js/components.js\" cross-origin=\"anonymous\" type=\"module\"></script>"
  ]

In [107]:
-- Basic CORS parameter definitions

{-# LANGUAGE OverloadedRecordDot #-}
{-# LANGUAGE RecordWildCards #-}
{-# LANGUAGE DerivingStrategies #-}
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE DeriveAnyClass #-}
{-# LANGUAGE OverloadedStrings #-}

module WebServer.CorsPolicy where

import Data.Aeson (FromJSON, ToJSON)
import Data.CaseInsensitive as CI
import Data.List (elem, lookup)
import Data.Maybe (fromMaybe)
import Data.Text (Text)
import qualified Data.Text as DT
import Data.Text.Encoding (encodeUtf8)
import GHC.Generics (Generic)
import Network.HTTP.Types.Header (hOrigin)
import Network.Wai
import Network.Wai.Middleware.Cors


data CorsConfig = CorsConfig {
   allowedOrigins :: [ Text ]
  , publicPrefixes :: [ Text ]
  , maxAge :: Maybe Int
  }
  deriving stock (Generic, Show)
  deriving anyclass (ToJSON, FromJSON)


defaultCorsPolicy = CorsConfig {
   allowedOrigins = [ "http://localhost", "http://localhost:7885" ]
  , publicPrefixes = [ "inlogin", "site", "/" ]
  , maxAge = Nothing
  }


setCorsPolicy :: CorsConfig -> Middleware
setCorsPolicy cConfig = cors $ \request ->
  if isSwaggerRequest request || isPublicApi request
  then Just $ simpleCorsResourcePolicy { 
    corsMethods = simpleMethods <> [ "OPTIONS" ]
    , corsRequestHeaders = simpleHeaders <> ["Authorization", "Content-Type"]
    }
  else Just $ CorsResourcePolicy { 
    corsOrigins = matchHostOrigin request
    , corsMethods = simpleMethods <> [ "DELETE", "OPTIONS" ]
    , corsRequestHeaders = ["Authorization", "Content-Type"]
    , corsExposedHeaders = Nothing
    , corsMaxAge = cConfig.maxAge
    , corsVaryOrigin = True
    , corsRequireOrigin = True
    , corsIgnoreFailures = False
    }
    where
      matchHostOrigin request = Just . fromMaybe ([], False) $ do
        origin <- lookup hOrigin (requestHeaders request)

        if CI.mk origin `elem` (mk . encodeUtf8 <$> cConfig.allowedOrigins)
        then return ([origin], True)
        else Nothing

      isSwaggerRequest request = case pathInfo request of
        []    -> False
        (x:_) -> "swagger" `DT.isPrefixOf` x

      isPublicApi request = case pathInfo request of
        []    -> False
        (x:_) -> any (`DT.isPrefixOf` x) cConfig.publicPrefixes


In [108]:
-- Wrap Json Web Keys for JWT in easy-to-use functions for main logic.

{-# LANGUAGE FlexibleContexts #-}

module WebServer.JWT (generateKeyPairIO, readJWK) where

import Control.Lens ((?~), (^.))
import Control.Monad.Except
import Crypto.JOSE.JWA.JWS (Alg (ES256))
import Crypto.JOSE.JWK (AsPublicKey (asPublicKey), Crv (P_256),
                                       JWK, JWKAlg (JWSAlg),
                                       KeyMaterialGenParam (ECGenParam),
                                       KeyOp (Sign, Verify), KeyUse (Sig),
                                       MonadRandom, genJWK, jwkAlg, jwkKeyOps,
                                       jwkUse)
import Crypto.JWT
import qualified Crypto.JWT as Jose
import Data.Aeson (eitherDecodeFileStrict, encodeFile)
import qualified Data.ByteString as BS
import qualified Data.ByteString.Lazy as BSL
import Data.Function ((&))
import Data.Text (Text)
import qualified Data.Text as DT

import Servant.Auth.Server (Auth, AuthResult (..), BasicAuth, BasicAuthCfg, CookieSettings (CookieSettings, cookieIsSecure)
                  , IsSecure (NotSecure), FromBasicAuthData, JWT, JWTSettings (validationKeys, audienceMatches), FromJWT (..), ToJWT (..), cookieIsSecure
                  , defaultCookieSettings, defaultJWTSettings, IsMatch (Matches, DoesNotMatch, Matches))


generateKeyPair :: MonadRandom m => m JWK
generateKeyPair = do
  k <- genJWK . ECGenParam $ P_256
  return $
    k
      & jwkAlg ?~ JWSAlg ES256
      & jwkKeyOps ?~ [Sign, Verify]
      & jwkUse ?~ Sig


-- | Generate jwk and public version according to kyrosid specs.
generateKeyPairIO :: FilePath -> IO JWK
generateKeyPairIO path = do
  jwk <- generateKeyPair
  let mbPubJWK = jwk ^. asPublicKey
  case mbPubJWK of
    Nothing -> fail "Public JWK generation error"
    Just pubJWK ->
      encodeFile (path <> ".pub") pubJWK
  encodeFile path jwk
  pure jwk


-- | Read JWK from file
readJWK :: FilePath -> IO JWK
readJWK path = do
  eJWK <- eitherDecodeFileStrict path
  case eJWK of
    Left e -> fail e
    Right jwk -> pure jwk


verifyJWT' :: (FromJWT a, VerificationKeyStore (ExceptT JWTError IO) (JWSHeader ()) ClaimsSet (IO JWKSet)) => JWTSettings -> BS.ByteString -> IO (Either Text a)
verifyJWT' jwtCfg input = do
  verifiedJWT <- liftIO $ runExceptT . withExceptT formJWTError $ do
    unverifiedJWT <- Jose.decodeCompact (BSL.fromStrict input)
    Jose.verifyClaims
      (jwtSettingsToJwtValidationSettings jwtCfg)
      (validationKeys jwtCfg)
      unverifiedJWT

  let eitherResult = verifiedJWT >>= decodeJWT

  return eitherResult
  where
    formJWTError :: JWTError -> Text
    formJWTError = DT.pack . show


jwtSettingsToJwtValidationSettings :: JWTSettings -> Jose.JWTValidationSettings
jwtSettingsToJwtValidationSettings s =
  defaultJWTValidationSettings (toBool <$> audienceMatches s)
  where
    toBool Matches = True
    toBool DoesNotMatch = False


In [109]:
-- Create some Runtime parameters for neater main logic.

module Options.Runtime (defaultRun, RunOptions (..), WebServerOptions (..)) where

import Data.Text (Text)

import WebServer.CorsPolicy (CorsConfig, defaultCorsPolicy)

data WebServerOptions = WebServerOptions {
    port :: Int
    , host :: Text
  }
  deriving (Show)

data RunOptions = RunOptions {
    debug :: Int
    , webServer :: WebServerOptions
    , jwkConfFile :: Maybe FilePath
    , corsPolicy :: Maybe CorsConfig
  }
  deriving (Show)

defaultRun :: RunOptions
defaultRun =
  RunOptions {
    debug = 0
    , webServer = WebServerOptions {
        port = 8181
        , host = "localhost"
      }
    , jwkConfFile = Nothing
    , corsPolicy = Nothing
  }


<div>
<h1>
Full Stack Deep Dive: What the deal with Haskell?    
</h1>
</div>

<div class="content flex py-2">
<div>
<div class="float-left rounded pt-4" style="padding: 5px 10px 0px 0px;"><img class="rounded" src="https://www.haskell.org/img/haskell-logo.svg" width="200"></div>
<p>is one of these languages that most senior developers have heard mysterious tales about. But as most stuff of legends, few people have had practical experience with. That's unfortunate as by today's state of technology having Haskell experience should be a standard practice for full-stack developers and CTOs.
</p>
<div class="float-right rounded"><img class="rounded-full shadow-md shadow-gray-700" style="margin: 0px 10px 0px 10px;" src="xstatic/img/lib_1.png" width="300" height="400"></div>
<div><p>I write this Jupyter notebook to address that issue while trying to take a little of your time as possible.</p>
<p>Why? I've come to realize that Haskell is the best tool I've used to reach the Zen of programming, the most likely to give me balance between theory and practice, quick prototyping and long-term software production, time-tested approaches and innovative techniques. That's not a conclusion I've jumped to; I have over 40 years of software development under my belt, during which I've worked with many many programming ecosystems in many many kind of software development projects with many many types of people. I've started using Haskell about 10 years ago, I slowly integrated its ecosystem into my normal work setup and especially over the last 5 years I've seen it mature to become an extremely powerful tool for today's requirements.
</p>
<p>To dig into important features of Haskell in the context of app development, this notebook doesn't go into setting up a development environment, compiling and running a first simple piece of code, and helping you going in your own environment. Instead I want to give a down-to-earth feeling of how, in the Haskell ecosystem, you produce a <em>normal, modern, efficient</em> web app, based on HTMX and Tailwind in the frontend, and with server-side rendering of templates filled with data provided by SQL queries, low-latency websocket communications... and security, distributed computing, 3rd party APIs, Javascript/C++/Python ecosystems integration, etc.
</p>
<p>If I manage to get you interested and you want to get more experience, it is easy to get the Haskell ecosystem on your local machine, read some of the many resources to learn how to write Haskell code with <a href="https://github.com/haskell/haskell-language-server"><i>language server</i></a> and <i>Copilot</i> support, and move forward in the creation of software through the <code>edit -- compile -- run -- observe</code> loop.
</p>
</div>
</div>
</div>

Why you should have some Haskell experience:
- you get to look at both theory of computing **and** everyday practicalities of software development, leading to a better thinking in general,
- best practices and first principles of programming are aplenty in the ecosystem, with substantial amount of high value-to-noise ratio packaged logic,
- you improve your ability to enounce clearly and efficiently complex design, in consistent layers of abstractions,
- you work in high-level of expressivenness providing short development time and minimal debugging efforts, yet you can get get high-performance execution for large scale computing comparable to **C** and **Go**,
- through its _domain specific languages_ (DSL) you can easily add your own language extensions for efficient handling of specific problems,
- through the meta-programming features you can build CI/CD that understands the code, embeds deployment logic within the build phases and achieve much tighter iterations of software development,
- the ecosystem is current with today's demand for interactive, media-rich, AI-enhanced cloud-based web applications (and most other buzz words you work with).

With all these great features, it's important to remember that:
- after over 30 years of existance, it's still a niche tool,
- advanced capabilities both in pure logic and meta-programming lead to amazing coding abstractions that use concepts living in ivory towers,
- documentation for getting onboarded and going on modern application development isn't adequate, and the community being made of many academic users writing in research paper style instead of dumb-down explanations often result in hard-to-reach information,
- its passionate community isn't mainstream, focusing more on fintech, super-computing and R&D than consumer-oriented applications.

I'll offer some thoughts on the state of the ecosystem and community at the end of the presentation.

<div class="grid justify-items-stretch bg-gray-700 hover:bg-orange-100 transition ease-in-out duration-150 rounded-md" style="margin: 0em 0em 2em 0em;">
  <img class="rounded-full justify-self-center animation-bounce" style="margin: 10px 8px 0px 0px;" src="xstatic/img/lib_2.png" width="250">
</div>
<div>

Some basic ideas about Haskell as a programming language and its main compiler, GHC:
- it has a simple and clean syntax: easy to learn, little decoratipns, high readability.
- it is functional: Haskell guts are [System F](!https://en.wikipedia.org/wiki/System_F), the foundation of high-order functional logic,
- it enforces immutability: a variable is assigned once (or *a term is bound to a name once only* the functional people would say),
- it has a strong support for pure logic without side effects: logic with the same inputs will always produce the same results, as theoritical computing expects would have it,
- it has a consistent support for practical computing: side effects and evolving execution contexts are required in real-life software, so that's also managed in the framework,
- it has a very powerful typing system that is also strongly enforced: anything is typed (even types themselves!), type matching must be exact for code to be accepted, there are types of types and type manipulations, etc,
- its typing system is powerful enough that it disappears from normal use: you write your code without types (a la JS) and Haskell figures out the rest (unlike JS!),
- it supports meta-programming: GHC contains a VM that can execute codes during compilation, leading to on-the-fly extra code generation, value derivation, interaction with the outside world, etc,
- it is extendable: new syntax can be created and added to Haskell to express domain-specific ideas, eg embedded SQL, and a powerful FFI provides an easy integration of existing C/C++ code,
- it is very optimizable: all kind of directives will drive the compiler to producte efficient machine code, the runtime to work with memory or garbage-collection constraints, as well as using Linear Types provide a way to execute code without garbage-collection (a la Rust).

Some basic ideas about Haskell as a runtime system:
- it puts an emphasis on compiled binaries, but you'll see in this notebook that it works just fine in interpreted mode,
- it provides a memory-management support (a garbage collector) that is very configurable,
- it is lazy: functions are not executed if they don't need to, which on the upside can provide significant optimizations, and on the downside create significant memory requirements,
- it is parallel: the concept of threading is built-in, and execution can happen in all kind of (safe) levels of parallelism depending on the underlying hardware,
- it is debuggable: the interpreter is the main way of debugging Haskell code, but GDB is also able to step through logic execution, 
- it is observable: execution performance is recordable, with statistics providing valuable ways to update source code for speeding up execution or managing resources better, and the heap can be introspected by a socket-connected debugger,
- it is self-contained: binaries can be statically linked, so a single executable file is all that's needed to run an application.

Ok, enough overview. But before moving on to writing a minmal web app, you need to be able to read Haskell code.

<div class="grid justify-items bg-gray-700 hover:bg-orange-100 rounded-md" style="padding:10px 0px 10px 0px">
  <img class="rounded-full justify-self-center" src="xstatic/img/soft_linlog_1.png" width="500">
</div>

The main idea with Haskell syntax is that in general you write:
* function applications: ```functionX argA argB```
* bindings: ```left_hand_side = right_hand_side```
* access to existing logic (packages): ```import AGreatPackage.BestModule (fctA, fctB)```
* name space creation (modules): ```module TopNamespace.SubNameSpace (exportA, exportB)```

Here we go... the baby code is a infix function application:

In [110]:
1 + 2

3

This is what one expects: the `+` function is applied to 1 and 2.  In other language + may be a reserved symbol, but in Haskell it's just another function defined somewhere. And any function can be called in prefix or infix mode, so the previous line is equivalent to:

In [111]:
(+) 2 3

5

Another typical example of baby code is the function application of the is-equal function:

In [112]:
1 == 2

False

That function returned the Boolean literal _False_ (the other literal being _True_).

Then there's the right side binding to the left side, which in other languages is typically an assignment statmement.  The baby code version of that is:

In [113]:
x1 = 2 + 3

That simply says that `x1` takes the value of the `2 + 3` function application.  Normally that means 6, but remember that Haskell is lazy, so deep down in the execution system it means that `x1` is going to be just `2 + 3` until `x1` needs to be used. Nothing useful in this case, but when the function application takes hour of CPU time to return a value or terabytes of RAM to perform, there will be none of these resources utilized until it's actually necessary to have a resulting value. By the way fpr any such situation you can tell the compiler that it should not enable laziness.

As Haskell is a functional language, remember that the previous statement is actually a function definition where `x1` takes no argument and works with constants.  In a pure immutable functional language, there are no variables, just function applications and term bindings...

The non-functional programmer will be more comfortable with the function definition:

In [114]:
incByOne arg_1 = 1 + arg_1

In this case, a function is defined which is called `incByOne` and which takes one argument.  When executed, it will add 1 to the `arg_1`, no surprise there.

The arguments that the function expects are simply listed one after the other, no need for any other decoration in the syntax. Here's a function that takes 2 arguments, and simply adds them together:

In [115]:
yetAnotherAdd arg_1 arg_2 = arg_1 + arg_2

Lambda notation is bread and butter in Haskell, so that function can be rewritten with the following syntax (which will trigger warnings from the compiler...):

In [116]:
aLambdaVersion = \arg_1 arg_2 -> arg_1 + arg_2

Line 1: Redundant lambda
Found:
aLambdaVersion = \ arg_1 arg_2 -> arg_1 + arg_2
Why not:
aLambdaVersion arg_1 arg_2 = arg_1 + arg_2Line 1: Avoid lambda
Found:
\ arg_1 arg_2 -> arg_1 + arg_2
Why not:
(+)

But lambda notation is not as clear to read, so the compiler will try to get you to use the first form.

Finally Haskell does all the expected tricks with curryfication, so it's possible to write:

In [117]:
addSynonym = (+)
addTwo = (+ 2)

addSynonym 2 3
addTwo 3

5

5

In that last case the compiler figures out that a function is a curried version of another and arrange arguments passed at function application in the right order without requiring to have them specified in the function definition.

Note that up to now we haven't seen any typing anywhere, just like in Javascript.  Given Haskell is strongly-typed and has a very powerful typing system, where are all the types?

Ok let's do use some typing notation.  First, a term can be given a specific type, like the `Int` type for the `2 + 3` calculation:

In [118]:
x1 = 2 + 3 :: Int

That tells the compiler that the right-hand-side value is of type `Int`.  The compiler will check that the `+` function and the `2` and `3` values all make sense, and then also give `Int` type to `x1`.  This is pretty obvious to anyone who has experience with typed languages. But given that Haskell typing system supports polymorphism, it means that `+` may exist for many different types (and it does!), and `2` and `3` literals may be representing different kind of intergers (16 bits, 32 bits, unbounded, ...). Adding ` :: Int` will lock in the meaning of `x1` and force its uses to be limited to `Int` applications.

**Important Note** Haskell enforces that type names must start with an upper-case letter, while variable and function names must start with a lower-case letter or a non-letter symbol such as `+`.

The typing information can also be provided in two parts; the first one gives the typing for the right-hand-side, while the second part gives the left-hand-side, which typing will need to match.  So the follwing means the same:

In [119]:
x1 :: Int
x1 = 2 + 3

That looks more like a good old C (and Java and ...) variable definition, where the variable is first declared and typed, and the value that it represents is given later and much match the type of the variable.

More typically one will provide typing for functions to documentent the meaning of the code and for locking in certain constraints on the arguments.

For example the following code clearly gives the intent of the developer on what the `showOneMore` function is expecting to work with (an `Int` argument), and what it will return (a `String` value. Here we once again use the 2 parts approach, where first the function is declared (with typing), and then its value is provided.  Now that is not how C (and friends) work for providing typing information to a function declaration, but if you think about it, the 2 parts syntax is simpler and more consistent...

In [120]:
showOneMore :: Int -> String
showOneMore arg_1 = show (1 + arg_1)

The typing for a function with two arguments is simply going to list them all, separated by arrows:

In [121]:
yetAnotherAdd :: Int -> Int -> Int
yetAnotherAdd arg_1 arg_2 = arg_1 + arg_2

Why the arrows as separator and the lack of difference between arguments and results in the typing syntax like other typed languages? The arrows are from the deep theoritical foundations of Haskell, in which lambda calculus has used that notation for ... a long time.  The lack of difference between arguments and result is again consistency and simplicity of syntax: a function that takes no argument is types as ` :: Result_Type` (like the `x1` seend before), so a function that takes an argument will add a type and arrow before that, which looks like `Arg_Type -> Result_Type`.

<section class="bar-graph bar-graph-horizontal bar-graph-one">
  <div class="bar-one">
    <span class="year">Progress</span>
    <div class="bar" data-percentage="70%"></div>
  </div>
</section>

At this point there's more than enough syntax knownledge to move on to the proverbial 'Hello World' example.  First, here's the code:

In [122]:
main :: IO ()
main =
  putStrLn "Hello World!!"

Haskell runtime follows the long tradition of the `main` function being the entry point for an application.
The function is typed as `IO ()`, the basic type for all things that deal with the real-life computing issue of input/output.
The `putStrLn` function takes a string (which is provided in literal form by using the double-quote character), and sends it to the _stdout_ of the running program.

In the Jupyter Notebook, we can simply call the function and the IHaskell kernel will redirect its *stdout* to be the output in the result cell:

In [123]:
main

Hello World!!

Moving forward, let's introduce a code management syntax: the `import` statement.  It instructs the compiler to go fetch code from some other place.  A simple concept in the good old days, but today it means resolving lots of data sources.  In Haskell, a unit of code is called a **module**, and it is a hierchical concept. Modules are stored in **packages**, which is the equivalent of libraries for C++, crates for Rust, packages (yeah!) for Javascript, and so on.  The GHC compiler came up early on with package management tools (*ghc-pkg*), and then more and more features have come up in that area over the years (*cabal*, *stack*, ...).  The main site for package distribution is [Stackage](https://stackage.org) but the largest site is [Hackage](https://hackage.haskell.org), which doesn't do as many efforts on curating the quality and stability of its content.

**Important Note** Haskell enforces that module names start with an upper-case letter, just like types.

Bringing in the entire content of a module in the current code namespace is the simplest form of `import`:

In [124]:
import Data.Char

A more advanced module inclusing will control what is brought in the current code namespace, and potentially add a prefix to what is brought in, as for:

In [125]:
import qualified Text.Blaze.Html5 as H
import qualified Text.Blaze.Html5.Attributes as A

We have added two modules to the current code namespace, and further more we've decided to have them accessable from now with the prefix *H* and *A*.

It turns out that these two modules are part of the [Blaze](https://hackage.haskell.org/package/blaze-html) package, which provides all HTML elements and attributes as functions with the same names.

To get there, we must take first a quick detour on another syntax element, the oh-so-powerful **comment**.  Comments in Haskell are written with prefixing with two dashes, or a by putting between curly brackets and dashes, such as:

In [126]:
-- Nothing impacts the compiler here...
{- and
  same
  here
-}

The GHC compiler has adopted a special version of curly-bracket comment to provide compiler directives (pragmas), and we'll use that at this point to add a very useful processing mode, the *Overloaded String* mode:

In [127]:
{-# LANGUAGE OverloadedStrings #-}

This is also feasible when using the *ghci* interpreter, which this notebook is using, by doing:

In [128]:
:set -XOverloadedStrings

The `OverloadedStrings` mode means that the compiler will do polymorphism resolution on the string literals.  That in turns means we can let the compiler figure out the right process to turn a string literal into a `Html` value that Blaze wants for most of its DOM building functions. With that available, we can now use Blaze functions to send some html into the notebook cells:

In [129]:
H.div H.! A.class_ "text-red-400 bg-gray-700" $ do
    H.p "This is an example of creating DOM elements similarly to JSX or (better) Elm..."
    H.div H.! A.class_ "rounded-md pb-3 pl-3" $ H.img H.! A.src "https://pbs.twimg.com/media/F6ezdRuWMAAVJfT?format=png&name=900x900" H.! A.width "150"
    H.b "Hello"

<div class="text-red-400 bg-gray-700">
    <p>
        This is an example of creating DOM elements similarly to JSX or (better) Elm...
    </p>
    <div class="rounded-md pb-3 pl-3">
        <img src="https://pbs.twimg.com/media/F6ezdRuWMAAVJfT?format=png&amp;name=900x900" width="150">
    </div>
    <b>
        Hello
    </b>
</div>

There are two more syntax elements introduced in this example that need to be covered before looking at the code in details.

The first one is the `$` symbol, which is a short-code for putting the rest of the line in parenthesis.
So `f1 $ 1 + 2` is the same as `f1 (1 + 2)`, and `f1 $ f2 $ 1 + 2` is the same as `f1 (f2 (1 + 2))`.

The second one is the `do` term, which is a short-hand notation for sequencing of a block of functions.  Function sequencing is a very long topic that brings in the famous *Monad* type, so we won't say more than it is similar to doing the `.then(...)` call in Javascript.
So

    do
      f1
      ...
      fN

means that first `f1` function will be called, and then `f2` will be called, and so on until the result of `fN` will be passed as the result of the whole `do` block.

Remember than parallelism is implied in Haskell, and without forcing sequencing the DOM elements could show up in any other that threading/optimization may find useful as they have no relationship amongst each other.

Now for some code inspection.  First note that all the HTML elements have an equivalent function in Blaze, that takes a `Html` value and returns a `Html` value.  The IHaskell kernel has a trick to automatically merges the DOM elements from that `Html` value into the notebook's result cell, and that's why we see actual rendered HTML rather than some other form of value presentation.  Without that trick, we could see

    <div class='text-red-400 bg-gray-700'><p>This ...

as a text literal, or maybe a JSON rendering of the entire `Html` content.

Second, this is a typical situation of learning curve risk.  On one side the Blaze package is very efficient and simple to use, two great qualities that usually don't go together. But to accomplish that, it uses advanced concepts (Monad, combinators, ...) and you'll be a mere user of the package unless you go through a steep learning curve of monoids, semigroups, functors, and so on.

Having said that, let's look at the code: a `div` function from the `Text.Blaze.Html5` module will generate the enclosing DOM element.  Further more a `style` function creates the attribute specification (red color), and the `!` function links `div` and `style`, and then receives the result of the `do` block.

In that block, each line calls a function that respectively generates *img*, *p* and *b* DOM elements, with some attributes (`!` calls) and content (the strings).

Rewriting this code without the module prefix gives:

    div ! class_ "text-red-400 bg-gray-700" $ do
    p "This is an example of creating DOM elements similarly to JSX or (better) Elm..."
    div ! class_ "rounded-md pb-3 pl-3" $ img ! src "https://pbs.twimg.com/..." ! width "150"
        b "Hello"


which is mure more readable, so that's what you will do in real code.  But in this notebook we'll continue to explicitely show which package provides which function with these prefixes, but in real life an editor with a Haskell language server (HLS) and no prefix would be a better way to go.

Using Blaze functions is very similar from writing template code for backend HTML generation.  But with all the advantages of Haskell language.

Now let's say we want to make some components a la React to build up an HTML page section in a modular way.  We'd go with something like:

In [130]:
import qualified Data.Text as T

-- define two components that take some parameters:
componentA width postfix = do
  H.p "This is an example of creating DOM elements similarly to JSX or (better) Elm..."
  H.img H.! A.src (H.textValue . T.pack $ "https://pbs.twimg.com/media/" ++ postfix) H.! A.width (H.textValue . T.pack . show $ width)

componentB someTxt color =
  H.b H.! A.class_ (H.textValue . T.pack $ "text-" ++ color) $ someTxt

-- then apply to get the main div by using the components:
H.div H.! A.class_ "text-red-400 bg-gray-700" $ do
  componentA 150 "F6ezdRuWMAAVJfT?format=png&name=900x900"
  componentB "Hello" "orange-200"



<div class="text-red-400 bg-gray-700">
    <p>
        This is an example of creating DOM elements similarly to JSX or (better) Elm...
    </p>
    <img src="https://pbs.twimg.com/media/F6ezdRuWMAAVJfT?format=png&amp;name=900x900" width="150">
    <b class="text-orange-200">
        Hello
    </b>
</div>

Fairly simple, and at this point you should be able to read most of the code.

A few more things are introduced in this example:
- the `++` function that concatenates two String values (it is an historical synonym for the more general concatenation function `<>`).
- the Data.Text module that provides a `String` to `Text` conversion function, `pack`. The `Text` type is a more C++-like implementation of strings, while the `String` type is a basic list implementation.
- the `.` operator, which concatenates functions, so `func1 . func2 $ a` means `func1 (func2 a))`.

Previously when using a *string* literal such as `"something"` (ie, a constant value that is text), the conversion sequence from a `String` value to a `Text` value to a `Html` value was done automatically by the compiler through the OverloadedStrings mode. But now we're dealing with variables values and the compiler doesn't do the work automatically. We use the form `H.textValue . t.pack $ <something>` that is more typical of *functional* code esthetics than its equivalent `H.textValue (t.pack <something>)`.  

We'll take a break from cool capabilities of the Blaze package and go back to introducing additional important syntaxic constructs.

The first one is the `case` construct, which is the workhorse of conditional logic. It looks like:

In [131]:
aValue = 2

case aValue of
  1 -> "one"
  2 -> "two"
  3 -> "three"
  _ -> "something different than 1, 2 or 3"
  

"two"

This kind of construct exists in most modern language, for example the *switch* in C and JS or the *match* in Rust.  The selector is on the right side of the arrow, the code to execute is on the left side of the arrow, and the `_` symbol matches anything so acts as the *default* selector.

By the way we're referring to *construct* instead of *statement* because it returns a value (like an expression and unlike a statement).

Haskell also provides the `if-then-else` construct, which is just a short-hand for writing:

    case <condition> of
      True -> <then-part>
      False -> <else-part>

So you can write that as `if <condition> then <then-part> else <else-part>`, and that construct returns either the result of the `<then-part>` or the `<else-part>` based on the True/False value of `<condition>`. Again that `if-then-else` construct will return a value, it is not a statement.

The third construct is the `let <something> in <something else>`.  Essentially the `let` construct split logic in multiple variable assignments, and then all these variables are re-assembled into a final result (the `in` part).  Recovering the previous HTML example, we could write: 

In [132]:
-- wrap up all the previous code into a new bigger component (function):
bigComponent halfWidth =
  -- first, compute all params
  let
    width = 2 * halfWidth
    urlPostfix = "F6ezdRuWMAAVJfT?format=png&name=900x900"
    helloString = "Hello"
    helloColor = "green-500"
  in
    -- then make the div element and its children:
    H.div H.! A.class_ "text-red-400 bg-gray-700" $ do
      componentA width urlPostfix
      componentB helloString helloColor

-- Invoke the overall html block:
bigComponent 75

<div class="text-red-400 bg-gray-700">
    <p>
        This is an example of creating DOM elements similarly to JSX or (better) Elm...
    </p>
    <img src="https://pbs.twimg.com/media/F6ezdRuWMAAVJfT?format=png&amp;name=900x900" width="150">
    <b class="text-green-500">
        Hello
    </b>
</div>

The main idea with the `let` construct is that it first provides a way to introduce names for a list of terms.  In an imperative language these would be consider variable declaration/assignements.  The new names are only visible in `let ... in ...` construct. Then these new terms will typically be assembled into a value in the `in` part, which becomes the result of the entire construct, as again we're not dealing with a statement.

<div class="grid justify-center">
  <img class="rounded-lg" src="xstatic/img/trumpet_1.png" width="200">
</div>

<h2>WHAT ABOUT THE TYPES!</h2>

Let's take a pause for a moment and note that there has been no explicit use of typing for any of the code except for actually showing off the typing notation.  A significant advantage of the Haskell language is that while it is (very) strongly-typed, the typing system is so powerful that you rarely need to bother with type annotation like one would do in C, Java or Rust. The compiler can figure out the logical typing arrangements and run code (idem for the interpreter); or if something is too vague it can alert the developer that the logic isn't sound and describe what isn't making sense.

Reflect on this a bit more...

It is a standard complaint about Haskell (and strongly-typed languages in general) that typing is a pain in the lower back to deal with.  It might be so in Java/TypeScript/Rust/etc... but in Haskell, one will be able to write code without typing like in Python and Javascript **AND** without all the errors of mistyped logic (and misspelled stuff).

In reality though you'll get use to add types (or let the editor/LSP do it for you) as a good way to document the code. Especially with LSP nowadays it's not so useful but writing types instead of logic just help thinking.

While you're doing some thinking, I'll borrow some html & css from the **Tailwind** site and make something more visually pleasing that the HTML I've been putting together up to now... Haskell language features used here will be discussed further down.

In [133]:
:set -XOverloadedRecordDot

import Control.Monad (mapM_, forM_)
import qualified Text.Blaze.Svg11 as S
import qualified Text.Blaze.Svg11.Attributes as Sa

data Project = Project {
    url :: T.Text
    , title :: T.Text
    , category :: T.Text
    , users :: [ UserPrj ]
  }

data UserPrj = UserPrj {
    name :: T.Text
    , unsplID :: T.Text
  }

projects = [
  Project {
    url = "proj_1"
    , title ="First Project"
    , category = "Testing"
    , users = [
      UserPrj "Joe A" "photo-1531123897727-8f129e1688ce"
      , UserPrj "Jane B" "photo-1494790108377-be9c29b29330"
    ]
  }
  , Project {
    url = "proj_2"
    , title ="Second Project"
    , category = "More testing"
    , users = [
      UserPrj "Achmed A" "photo-1494790108377-be9c29b29330"
      , UserPrj "Simona B" "photo-1506794778202-cad84cf45f1d"
    ]
  }
 ]

sectionD =
  H.section $ do
    H.header H.! A.class_ "bg-white space-y-4 p-4 sm:px-8 sm:py-6 lg:p-4 xl:px-8 xl:py-6" $ do
      H.div H.! A.class_ "flex items-center justify-between" $ do
        H.h2 H.! A.class_ "font-semibold text-slate-900" $ H.toHtml "Projects"
        H.a H.! A.href "javascript:void()" H.! A.class_ "hover:bg-blue-400 group flex items-center rounded-md bg-blue-500 text-white text-sm font-medium pl-2 pr-3 py-2 shadow-sm" $ do
          S.svg S.! Sa.width "20" H.! Sa.height "20" H.! Sa.fill "currentColor" H.! A.class_ "mr-2" $
            S.path S.! Sa.d "M10 5a1 1 0 0 1 1 1v3h3a1 1 0 1 1 0 2h-3v3a1 1 0 1 1-2 0v-3H6a1 1 0 1 1 0-2h3V6a1 1 0 0 1 1-1Z"
          H.toHtml "New"
      form
    projectsD projects

form =
  H.form H.! A.class_ "group relative" $ do
      S.svg S.! Sa.width "20" S.! Sa.height "20" S.! Sa.fill "currentColor" S.! 
            A.class_ "absolute left-3 top-1/2 -mt-2.5 text-slate-400 pointer-events-none group-focus-within:text-blue-500" $
        S.path S.! Sa.fillRule "evenodd" S.! Sa.clipRule "evenodd"
            S.! Sa.d "M8 4a4 4 0 100 8 4 4 0 000-8zM2 8a6 6 0 1110.89 3.476l4.817 4.817a1 1 0 01-1.414 1.414l-4.816-4.816A6 6 0 012 8z"
      H.input H.! A.class_ "focus:ring-2 focus:ring-blue-500 focus:outline-none appearance-none w-full text-sm leading-6 text-slate-900 placeholder-slate-400 rounded-md py-2 pl-10 ring-1 ring-slate-200 shadow-sm"
        H.! A.type_ "text" H.! A.placeholder "Filter projects..."

-- A components for showing projects
projectsD projects =
  H.ul H.! A.class_ "bg-slate-50 p-4 sm:px-8 sm:pt-6 sm:pb-8 lg:p-4 xl:px-8 xl:pt-6 xl:pb-8 grid grid-cols-1 sm:grid-cols-2 lg:grid-cols-1 xl:grid-cols-2 gap-4 text-sm leading-6 dark:bg-slate-900/40 dark:ring-1 dark:ring-white/5" $
    mapM_ aProjectD  projects

aProjectD aProj =
  H.li H.! A.class_ "group cursor-pointer rounded-md p-3 bg-white ring-1 ring-slate-200 shadow-sm hover:bg-blue-500 hover:ring-blue-500 hover:shadow-md dark:bg-slate-700 dark:ring-0 dark:highlight-white/10 dark:hover:bg-blue-500 hidden sm:block lg:hidden xl:block" $
    H.a H.! A.href (H.textValue aProj.url) H.! A.class_ "hover:bg-blue-500 hover:ring-blue-500 hover:shadow-md group rounded-md p-3 bg-white ring-1 ring-slate-200 shadow-sm" $
      H.dl H.! A.class_ "grid sm:block lg:grid xl:block grid-cols-2 grid-rows-2 items-center" $ do
        H.div $ do
          H.dt H.! A.class_ "sr-only" $ H.toHtml "Title"
          H.dd H.! A.class_ "group-hover:text-white font-semibold text-slate-900" $ H.toHtml aProj.title
        H.div $ do
          H.dt H.! A.class_ "sr-only" $ H.toHtml "Category"
          H.dd H.! A.class_ "group-hover:text-blue-200" $ H.toHtml aProj.category
        H.div H.! A.class_ "col-start-2 row-start-1 row-end-3 sm:mt-4 lg:mt-0 xl:mt-4" $ do
          H.dt H.! A.class_ "sr-only" $ H.toHtml "Users"
          H.dd H.! A.class_ "flex justify-end sm:justify-start lg:justify-end xl:justify-start -space-x-1.5" $
            forM_ aProj.users (\u -> H.img H.! A.src (fullUnsplashUrl u.unsplID) H.! A.alt (H.textValue u.name) H.! A.class_ "w-6 h-6 rounded-full bg-slate-100 ring-2 ring-white")
  where
  fullUnsplashUrl anID =
    H.textValue $ "https://images.unsplash.com/" <> anID <> "?auto=format&fit=facearea&facepad=2&w=48&h=48&q=80"

demoPage aReason = 
  H.html $ do
    H.head $ do
      H.link H.! A.href "https://unpkg.com/tailwindcss@^1.0/dist/tailwind.min.css" H.! A.rel "stylesheet"
      H.link H.! A.href "xstatic/css/pack_1.css" H.! A.rel "stylesheet"
      H.link H.! A.href "xstatic/css/pack_2.css" H.! A.rel "stylesheet"
    H.body H.! A.class_ "dark antialiased text-slate-500 dark:text-slate-400 dark:bg-slate-900" $
      H.div H.! A.style "color: red; background: rgb(115 120 128)" H.! A.class_ "lg:col-span-5 xl:col-span-6 flex flex-col" $
        H.div H.! A.class_ "relative z-10 rounded-xl bg-white shadow-xl ring-1 ring-slate-900/5 overflow-hidden my-auto xl:mt-18 dark:bg-slate-800" $
          H.div H.! A.class_ "container mx-auto p-4" $ do
            H.h1 H.! A.class_ "text-2xl font-bold mb-4" $ H.toHtml aReason
            sectionD

demoPage "Just taking a break"

<html>
    <head>
        <link href="https://unpkg.com/tailwindcss@^1.0/dist/tailwind.min.css" rel="stylesheet">
        <link href="xstatic/css/pack_1.css" rel="stylesheet">
        <link href="xstatic/css/pack_2.css" rel="stylesheet">
    </head>
    <body class="dark antialiased text-slate-500 dark:text-slate-400 dark:bg-slate-900">
        <div style="color: red; background: rgb(115 120 128)" class="lg:col-span-5 xl:col-span-6 flex flex-col">
            <div class="relative z-10 rounded-xl bg-white shadow-xl ring-1 ring-slate-900/5 overflow-hidden my-auto xl:mt-18 dark:bg-slate-800">
                <div class="container mx-auto p-4">
                    <h1 class="text-2xl font-bold mb-4">
                        Just taking a break
                    </h1>
                    <section>
                        <header class="bg-white space-y-4 p-4 sm:px-8 sm:py-6 lg:p-4 xl:px-8 xl:py-6">
                            <div class="flex items-center justify-between">
                                <h2 class="font-semibold text-slate-900">
                                    Projects
                                </h2>
                                <a href="javascript:void()" class="hover:bg-blue-400 group flex items-center rounded-md bg-blue-500 text-white text-sm font-medium pl-2 pr-3 py-2 shadow-sm">
                                    <svg width="20" height="20" fill="currentColor" class="mr-2">
                                        <path d="M10 5a1 1 0 0 1 1 1v3h3a1 1 0 1 1 0 2h-3v3a1 1 0 1 1-2 0v-3H6a1 1 0 1 1 0-2h3V6a1 1 0 0 1 1-1Z" />
                                    </svg>
                                    New
                                </a>
                            </div>
                            <form class="group relative">
                                <svg width="20" height="20" fill="currentColor" class="absolute left-3 top-1/2 -mt-2.5 text-slate-400 pointer-events-none group-focus-within:text-blue-500">
                                    <path fill-rule="evenodd" clip-rule="evenodd" d="M8 4a4 4 0 100 8 4 4 0 000-8zM2 8a6 6 0 1110.89 3.476l4.817 4.817a1 1 0 01-1.414 1.414l-4.816-4.816A6 6 0 012 8z" />
                                </svg>
                                <input class="focus:ring-2 focus:ring-blue-500 focus:outline-none appearance-none w-full text-sm leading-6 text-slate-900 placeholder-slate-400 rounded-md py-2 pl-10 ring-1 ring-slate-200 shadow-sm" type="text" placeholder="Filter projects...">
                            </form>
                        </header>
                        <ul class="bg-slate-50 p-4 sm:px-8 sm:pt-6 sm:pb-8 lg:p-4 xl:px-8 xl:pt-6 xl:pb-8 grid grid-cols-1 sm:grid-cols-2 lg:grid-cols-1 xl:grid-cols-2 gap-4 text-sm leading-6 dark:bg-slate-900/40 dark:ring-1 dark:ring-white/5">
                            <li class="group cursor-pointer rounded-md p-3 bg-white ring-1 ring-slate-200 shadow-sm hover:bg-blue-500 hover:ring-blue-500 hover:shadow-md dark:bg-slate-700 dark:ring-0 dark:highlight-white/10 dark:hover:bg-blue-500 hidden sm:block lg:hidden xl:block">
                                <a href="proj_1" class="hover:bg-blue-500 hover:ring-blue-500 hover:shadow-md group rounded-md p-3 bg-white ring-1 ring-slate-200 shadow-sm">
                                    <dl class="grid sm:block lg:grid xl:block grid-cols-2 grid-rows-2 items-center">
                                        <div>
                                            <dt class="sr-only">
                                                Title
                                            </dt>
                                            <dd class="group-hover:text-white font-semibold text-slate-900">
                                                First Project
                                            </dd>
                                        </div>
                                        <div>
                                            <dt class="sr-only">
                  

Ok, not as good as the original, but still really cool to get that kind of result directly into Jupyter's cells. Doing some thinking of our own, when looking at the CSS classes in the code, we feel spending time understanding Haskell is more rewarding than spending doing the same on stylesheet rules...

Introducing new data structures is a typical task of programming; in Haskell the most basic construct for introducing a new structure, which is also a new type, is `data`.  For example, one would define a new structure to track image size in the following way:

In [134]:
data ImageSize_1 = ImgSz_1 Int Int

which means that a value of the new type called `ImageSize` is the assembly of two values of type `Int`, and such a `ImageSize_1` value is identified by the _value constructor_ `ImgSz`.

In real life the value construtor is normally the same term as the type name, as

    data ImageSize_1 = ImageSize_1 Int Int

but that's just a convention and it can be anything that starts with an upper-case letter.

**Note** that using `data` for types instead of `type` is a bit weird; but unfortunately Haskell designers chose the keyword `type` to do type aliasing.

Now to create a value of type `ImageSize_1`, one would write:

In [135]:
isz_1 = ImgSz_1 200 300

But that's makes rather uninformative code; it would be nicer to know immediately which of the two number is the width and which is the height. To solve this lack of information, we can use the classic record notation.  It's classic, but up to 2021 it wasn't part of Haskell at all (!!), and it still require the *OverloadedRecordDot* mode to be activated:

In [136]:
:set -XOverloadedRecordDot

data ImageSize_2 = ImgSz_2 { width :: Int, height :: Int }

And then to create a value of type `ImageSize_2` with more explicit information, one would write:

In [137]:
isz_2 = ImgSz_2 { width = 200, height = 300 }

With the record notation, recoving components values of the type is standard code:

In [138]:
if isz_2.width < 200 || isz_2.height > 500 then "Out of bound" else "No problemo"

"No problemo"

We may be old school (Prolog anyone?) and want to use the ImageSize_1 type definition to inspect its content.  We can use pattern matching to do so:

In [139]:
ImgSz_1 width height = isz_1
if width < 200 || height > 500 then "Kaboom" else "All good"

"All good"

What happened just now is that the `ImgSz_1 <v1> <v2>` is a pattern that will match a value of type `ImageSize_1`.  When there's binding between such a pattern and a value, the `<v1>` and `<v2>` terms are also bound to the internal components of the value.

While it's old school, it turns out to be an elegant way to write code.  It used to be a big thing for writing expert systems in Prolog, and in Haskell it is a popular way to write conditional code that turns out to be very readable, as we'll see with more complex type definitions.

While we are here, the notation for modifying values from a record structure is:

In [140]:
isz_3 = isz_2 { height = 500 }
"Width: " <> show isz_3.width <> ", height: " <> show isz_3.height

: 

The result of that line of code is a new `ImageSize_2` value with the same `height` as `isz_2` but with a `width` of 500.  As Haskell enforces immutability, `isz_2.width = 500` isn't allowed.

In the academia the kind of type we have defined with `ImageSize_<x>` is called a multiplicative type; normal people will simply call it a record, or class, or *struct*. It comes from the idea that it spans all possible values of each component, or again, the total amount of values is the multiplication of each quantity of values for each component in the type.

By contrast, a summative type presents a serie of alternatives for its definition, and thus the total amount of values for the type is the sum of total values for each alternative definitions.  That's known as an *union* in C, *enum* in Rust, etc.

Creating a summative type in Haskell is done by putting a list of type definitions, seperated by `|` symbol (the alternative, *or*):

In [141]:
data Sizing =
  Image ImageSize_2            -- expressing the size for an image based on previous definition
  | Video Int Int Int Double   -- expressing the size based on some video concepts
  | Unknown String             -- being able to express some other wild size parameters.

Then we can create some values of type Sizing by writing:

In [142]:
-- Sizing for an image:
imgSz = Image isz_2
-- Sizing for a video:
vidSz = Video 1920 1080 32 29.9
-- Sizing somethign we don't know about:
unkSz = Unknown "1,2,3 and 4,5,6, and let's say big, big"

At that point, let's revisit the `case` construct and see how it's typically used:

In [143]:
someSize = vidSz

case someSize of
  Image imgSize -> "width: " <> show imgSize.width <> ", height: " <> show imgSize.height
  Video w h _ _ -> "width: " <> show w <> ", height: " <> show h
  Unknown _ -> "Who knows!"

"width: 1920, height: 1080"

This shows how pattern matching and summative types makes for readable code (but not necessarely clever or useful code...).

Similarly, pattern matching and function defininition can provide a clearer way of writing code. In the following example function `f1` is defined using pattern matching, and the compiler will be clever enough to assemble all the definitions and get the right one to be selected at runtime.

In [144]:
-- create a synonym of the string conversion combined function for more readable code later on:
useString = H.toMarkup . T.pack

-- define a function that works with Image values:
f1 (Image imgSize) = do
    H.b "IMAGE"
    H.ul $ do
      H.li $ useString $ "width: " <> show imgSize.width
      H.li $ useString $ "height: " <> show imgSize.height

-- define a function that works with Video values:
f1 (Video w h _ _) = do
    H.b "VIDEO"
    H.ul $ do
      H.li $ useString $ "width: " <> show w
      H.li $ useString $ "height: " <> show h

-- define a function that works with Unknown values:
f1 (Unknown _) = H.b "Who knows!"

-- apply the f1 function to a value of type Sizing:
f1 vidSz

<b>
    VIDEO
</b>
<ul>
    <li>
        width: 1920
    </li>
    <li>
        height: 1080
    </li>
</ul>

Tricky question: why use

    H.li $ useString $ "width: " <> show imgSize.width

instead of

    H.li useString "width: " <> show imgSize.width

The Haskell language does parameter application left-to-right, so in the second case it uses `useString` as the parameter for `H.li`, and then try to apply the resulf of that to `"width: "`, and so on.  So to get the desired result, first we have to combine everything after `H.li` into a single value by inserting a `$`, then combine everything after the useString into a single value by inserting another `$`.  For the rest of the line, we have an infix function (`<>`) that takes 2 arguments, which are `"width: "` and `show imgSize.width`, so at that point we're good as they're both `String` values.


We have so far covered a lot of ground, but we still have managed to stay away from a very important type in programming language: the list!  Haskell being a functional language, the list concept is very important as it's the basis for iteration, which other languages implement as the *for-loop* or *do-while* statements.

The very first thing about a list in Haskell is that it's written as array in most other languages:

In [145]:
aList = [ 1, 2, 3, 4, 5 ]

but it's not an array, it's a **list**. Think *Lisp*, ie single linked list...
That means there's no index to get elements, instead one has to use the [*car : cdr*](https://en.wikipedia.org/wiki/CAR_and_CDR) technique.
The `:` function is used to add a new element to an existing list, as in:

In [146]:
6 : aList

[6,1,2,3,4,5]

And conversely, pattern matching with `:` is used to extract the first element of a list:

In [147]:
firstElement : restOfList = aList
firstElement

1

Adding more entries at the end of a list is a concatenation, and as seen before that's done with the `<>` function.  The `String` type is actually just a synonym for the `[Char]` type, ie the basic string is a list of `Char` values (full Unicode supported).

In [148]:
aList <> [ 6 ]
"allo" <> " byebye"

[1,2,3,4,5,6]

"allo byebye"

Developers that don't have a formal computer science will find it weird to use lists for all kind of iterative and recursive flow control, but (hopefully) those with the formal background will remember that they used that approach in all kind of algorithms instead of the lowly index scans.

The typical iterations and recursions on list are done with either the `map` and `fold` family of functions, or using `case` construct with pattern matching on the *car : cdr* values.  For example:

In [149]:
Prelude.map (H.b . useString . show) aList      -- apply a function to each element of aList

Prelude.foldl (*) 2 aList   -- multiply recursively each value of the list and double that.

showOff inList = H.ul $ fRecur inList

-- Call itself recursively, but operate as an iterator over a list of values.
fRecur inList =
  case inList of
    [] -> H.b "-- 0 --"        -- Edge case, the list is empty.
    [car] -> itemize car       -- The list is a single element.
    car : cdr -> itemize car <> fRecur cdr       -- There are more than 1 element in the list, call itself to iterate over all elements.
  where
    itemize = H.li . useString . show

showOff aList

fRecur []

<b>
    1
</b>

<b>
    2
</b>

<b>
    3
</b>

<b>
    4
</b>

<b>
    5
</b>

240

<ul>
    <li>
        1
    </li>
    <li>
        2
    </li>
    <li>
        3
    </li>
    <li>
        4
    </li>
    <li>
        5
    </li>
</ul>

<b>
    -- 0 --
</b>

This example introduces the `where` construct, which is kind of a mirror of the `let` construct as it lets us provide a set of definitions **after** the main logic rather than before.  There's no impact of code execution sequencing in comparison to `let`, it's just a convenience for writing a set of expressions after the fact instead of before.
Also instead of adding a variable definition, we have a here function definition.  At the end of the day, variables are just functions with 0 argument, so defining functions or variables is exactly the same thing, both in the `let` and `where` syntax.

Ok so we've covered a very important part of functional concepts, the list and the iteration and recursion build on it.

And except where we explicitely wanted to talk about typing, still no typing to bother with! <span>&#128512;</span>

<h2>Fetching web data</h2>
Moving on to more complex computing and Haskell concepts.
First, we'll look at HTTP requests, the basic communication protocol of the web.

I'll use a simple REST api provider, the USA's [Car Vehicle API](https://vpic.nhtsa.dot.gov/api/), to fetch information about vehicles used in the USA.  The API will return JSON data, which will be upgraded to Haskell data structures automagically by logic from the [Aeson](https://hackage.haskell.org/package/aeson) package.

The basic HTTP tool of Haskell, the [Network.HTTP.Client](https://hackage.haskell.org/package/http-client-0.7.14/docs/Network-HTTP-Client.html) package, does the job easily.

Also to illustrate a point about monadic stuff, we'll put the logic in a toy function.

In [150]:
import qualified Network.HTTP.Client as N
import qualified Network.HTTP.Client.TLS as N

vehicleFetch query =
  let
    url = "GET https://vpic.nhtsa.dot.gov/api/vehicles/" <> query
    request = N.setQueryString [("format", Just "json")] $ N.parseRequest_ url
    settings = N.tlsManagerSettings
  in do
    reqMgr <- N.newManager settings
    N.httpLbs request reqMgr
 
response <- vehicleFetch "getallmakes"

The function is split in two parts using a `let` construct.  In the first one, the `url`, `req` and `settings` values are specified.
In the second part the HTTP fetch happens. Fetching data on the web is far from being a _pure_, always repeatable operation. In Haskell that means computations will be wrapped in one of those famous monads, the `IO`. There is no indication of the `IO` type being used in the code as it is all deduced by the compiler. But the two _impure_ computations need to be performed in a strict order, and that's why the `do` keyword shows up in the `in` part of the the `let` construct.
Once results are generated within the `IO` monad, we'll want to get them back to the _pure_ world, which is done by using the `<-` (left-pointing arrow) symbol instead of `=` to extract the values from the `IO` _impure_ *box* to _pure_ world.
The first _impure_ operation is the definition of a HTTP communication manager; it is a minor overhead due to using this low-level library that exposes the connection layer.
The next operation is the actual fetching of data, done by the `httpLbs` function.  This being the last value encountered in the `vehicleFetch` function, it also become the result it will return. That means that `vehicleFetch` returns _impure_ values embedded in the `IO` monad.

If we were to specify typing for that function, it would gain:

    vehicleFetch :: String -> IO (N.Response Data.ByteString)

meaning that the function takes a `String` parameter, and returns a `ByteString`-based (yet another string implementation!) HTTP `Response` wrapped in the `IO` type.

But why bother?

Finally to get the data received from the web after invoking the `vehicleFetch` out of the `IO` *box* and back in the _pure_ world, the `<-` symbol is used again.
If the Internet is working properly, the `response` value is a 200 HTTP reply with a body full of JSON data.
First, we'll take a peek at the status and first 150 characters of the body, just to have an idea of what's inside.

In [151]:
import qualified Data.ByteString.Lazy as LBS
import qualified Data.Text.Lazy.Encoding as TL
import qualified Data.Text.Lazy as TL

print $ "status: " <> show response.responseStatus <> ", body: " <> (Prelude.take 150 . TL.unpack . TL.decodeUtf8 $ response.responseBody)

"status: Status {statusCode = 200, statusMessage = \"OK\"}, body: {\"Count\":11086,\"Message\":\"Response returned successfully\",\"SearchCriteria\":null,\"Results\":[{\"Make_ID\":4877,\"Make_Name\":\"1\\/OFF KUSTOMS, LLC\"},{\"Make_I"

That last `print` call should show you a 200 status code, and then a body where you can see the beginning of a long JSON payload, with the backslashes escaping quotes in this strings-in-a-string value.
That little peek operation turns out to be a typical example of the string conversion work that is frequent in Haskell. Due to the many approaches to encoding string data (for scanning, for compactness, for high-speed string manupilations, etc), there's often string transmutation operations to do. Simple but annoying... the precision at which you can control string management gets the better part of the annoyment and you quickly get used to it. In this case, 3 typical packages are imported and then the `unpack` and `decodeUtf8` functions get the job done.

Assuming the HTTP response is a 200, then we can move to upgrading the JSON string to Haskell data structures.

<h2>Using JSON (and meta-programming)</h2>

Converting from/to JSON and Haskell values is typically done *automagically*, but it's really worth looking at how it happens, as it is an introduction to Haskell meta-programming.

The `GHC.Generics` core Haskell package together with the `DeriveGeneric` and `DeriveAnyClass` modes are the basic setup to manipulate code with code during compilation.
While in some modern compilers, such as **Go**, it is possible to transform a JSON string into a `struct` and vice-versa, the mechanics of that are hidden within the compiler itself and some special libraries.  In Haskell, these mechanisms are meta-programming code that can extend or modify at will existing code.  It is a blessing for automatizing the generation of boiler-plate code and doing all kind of further code generation from a few lines provided by the developer.

In this case, the very popular JSON package [Aeson](https://hackage.haskell.org/package/aeson) has logic to introspect a type (data structure) and then generate automatically at compile-time the logic to parse JSON strings into a value of that given type, and conversily to generate a JSON representation of from value of that type.

For example:

In [152]:
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE DeriveAnyClass #-}

import GHC.Generics
import qualified Data.Aeson as J

data SimpleThing = ST {
    aField :: String
    , anotherField :: Int
  }
  deriving (Show, Generic, J.FromJSON, J.ToJSON)

The `SimpleThing` type is a simple container for a `String` and an `Int`. The interesting part is the `deriving` keyword that follows.
That instruts the Haskell compiler to generate all the required code to convert a value into a string reprsentation (`Show`), to create a introspective representation of the type (`Generic`), and finally to create conversion logic to go back and forth between a value of `SimpleThing` and its JSON representation (`FromJSON`, `ToJSON`).
None of that generated code shows up, but the compiler will remember it's there and will use it whenever required.

Now let's create a value of `SimpleThing` type, then print it and both encode it to and decode it from the JSON representation, a very standard operation in web applications today.

In [153]:
simpleThing = ST { aField = "first value", anotherField = 100 }

-- Use the Show code to convert the value to a string.
print simpleThing

-- Use the ToJSON code to get a JSON string out of the value.
J.encode simpleThing

-- Parse the JSON string to get a SimpleThing value.
simpleJson = "{\"aField\":\"first value\",\"anotherField\":100}"
show (J.decode simpleJson :: Maybe SimpleThing)

ST {aField = "first value", anotherField = 100}

"{\"aField\":\"first value\",\"anotherField\":100}"

"Just (ST {aField = \"first value\", anotherField = 100})"

The `encode` function is simply going to apply the custom-made conversion logic for a `SimpleThing` value, which results in a JSON string as we'd expect.

Then the `decode simpleJson` function is asked to be of type `Maybe SimpleThing`.  That instructs the compiler to apply the custom-made conversion logic in the execution of the `decode` to do the JSON string parsing.
The `Maybe` type container is Haskell way of managing optional results, similarly to the `?` symbol in **Rust** or **TypeScript**, which does away with the `null` issues of so many other languages.  It is an additive type, which uses two constructors: `Nothing` and `Just <a value>`. So when a function can either emit a result or return a void, we'll define it as a `Maybe <some type>` function.

In this example the `decode` function does work and thus returns a `Just SimpleThing` value, which is then shown as a string.

The way *Aeson* meta-programming logic is available for extension/modification is especially important. Unfortunately you'll have to discover that on your own later on as we need to keep moving, although we'll do use a bit of that to massage the data we've received.

Going back to the `vehicleFetch` example, we can now build the `ResponseVehicle` and `CarDescription` structures (types) to hold the data returned in JSON format.

In real life use of 3rd party APIs, it's not always neat and simple to parse the JSON fields.  In the case of the *Vehicle API*, the top-level JSON objects fields start with an uppercase letter, and then the individual car description object has a 'Make_' prefix for its fields that's redundant.  On the other side, Haskell fields in a record must start with a lowercase letter, and we don't want to have to carry the 'make_' prefix in our code.

To deal with that, we'll inject a few modifications to the default meta-programming logic that deals with the conversion of JSON representations, using the `instance` keyword instead of the `deriving` one for creating the FromJSON code.  And yes, `instance` means *instance of a class* as Haskell as a *class* concept, which isn't object-oriented but means that types can be grouped to use common base logic which can be extended as required for meta-programming purposes.  It's yet another very interesting aspect of Haskell!

Here's the code to represent the vehicle info in Haskell as a `ResponseVehicle` and `CarDescription` types, and parse the respective JSON representations:

In [154]:
import qualified Data.Char as C

-- Specify the structures for holding the API overall reply and each car specification.
data ResponseVehicle = ResponseVehicle {
    count :: Int
    , message :: String
    , searchCriteria :: Maybe String
    , results :: [ CarDescription ]
  }
  deriving (Show, Generic)

data CarDescription = CarDescription {
    uid :: Int
    , name :: T.Text
  }
  deriving (Show, Generic)


-- Extend the JSON-conversion logic of Aeson with specific logic for the Vehicle API format:
instance J.FromJSON CarDescription where
  parseJSON (J.Object obj) = CarDescription <$> obj J..: "Make_ID" <*> obj J..: "Make_Name"

instance J.FromJSON ResponseVehicle where
    parseJSON = J.genericParseJSON J.defaultOptions { J.fieldLabelModifier = firstUpper }

-- Define a 'make first letter upper-case' helper:
firstUpper (c:cs) = C.toUpper c : cs
firstUpper [] = []

The *Aeson* package provides the basic meta-programming for conversion from JSON to a Haskell value in the `parseJSON` function. In the code above, the `instance J.FromJSON CarDescription where` is telling the compiler that we'll add to the existing meta-programming logic of *Aeson*'s `FromJSON` class of logic (thus the use of *instance*, but it's not related to object-oriented programming). Then we provide a definition for `parseJSON` that builds a `CarDescription` value from the `Make_ID` and `Make_Name` fields in the JSON.

The somewhat cryptic `<$>` and `<*>` symbols are simply helper functions that traverse the *boxes* the JSON values are stored in by *Aeson* logic.  They are worth looking into a bit more.

A simple example of using these helpers for doing math with values that are within the `Maybe` container is:

In [155]:
(+) <$> Just 2 <*> Just 5

Just 7

The `<$>` and `<*>` have pushed the addition to the integers that are within a `Just` *box*, and then the result is packaged back into the same kind of box, ie a `Just`.  The neat thing is that those helpers follow the rules of the container, so in the case of:

In [156]:
(+) <$> Just 4 <*> Nothing

Nothing

The addition doesn't crash because there's no value to add the 4 with and instead the result is `Nothing`, which makes a lot of sense.  That's because the rules of the `Maybe` type are enforced, something the addition function knows nothing about.

The weird `J..:` syntax is *Aeson*'s field value extractor function `.:` prefixed the package `J.`.  As noted earlier on, functions are everywhere and there are a just a few syntax rules in Haskell, so exotic function names such as `.:`, or `!` for Blaze, abound. And the decision to use of package prefix everywhere in this notebooks is biting us a bit now.

Going back to the `parseJSON` function extended for the `ResponseVehicle` type, the variation applied takes a completely different form. Instead of providing new extraction logic, the configuration parameters of *Aeson*'s built-in `genericParseJSON` function are modified: the `fieldLabelModifier` of the `defaultOptions` value is updated with a function that maps the first character of each field name to an uppercase value. That's it.

Getting the JSON data returned by the Vehicle API converted into proper Haskell values then becomes a simple `J.decode` call; the on-the-fly generated logic takes care of field mapping, sub-objects resolution, array parsing, etc. Further more, if you were to look at the generated code (it's possible to ask the compiler to spit it out), you'd see it is fast and efficient because it's fine-tuned for each kind of JSON conversion required. For example we used the `Text` type for the `name` field in the `CarDescription`, which is a more efficient string implementation than `String`, and thus the generated JSON parsing logic for that perticular kind of string will also be used in the `CarDescription` JSON translation logic.

In the next block of code, we bring back Blaze to get HTML beautification around the JSON to Haskell value conversion to show how we typically deal with `Maybe` situations, and how simple it is to preview values of complex types.

In [157]:
showCar aDescription = do
  H.b . useString . T.unpack $ aDescription.name
  H.i . useString $ " (id: " <> show aDescription.uid <> ")"

mbApiResponse = J.decode response.responseBody :: Maybe ResponseVehicle

H.div $ case mbApiResponse of
  Nothing -> H.b "No result."
  Just result ->
    let
      count = result.count
    in
      if Prelude.length result.results == 0
        then H.b "Nothing???"
        else do
          H.b . useString $ "Got " <> show count <> " descriptions, the first few ones are:"
          H.br
          H.ul $ mapM_ (H.li . showCar) (Prelude.take 5 result.results)


<div>
    <b>
        Got 11086 descriptions, the first few ones are:
    </b>
    <br>
    <ul>
        <li>
            <b>
                1/OFF KUSTOMS, LLC
            </b>
            <i>
                 (id: 4877)
            </i>
        </li>
        <li>
            <b>
                102 IRONWORKS, INC.
            </b>
            <i>
                 (id: 11257)
            </i>
        </li>
        <li>
            <b>
                12832429 CANADA INC.
            </b>
            <i>
                 (id: 12255)
            </i>
        </li>
        <li>
            <b>
                17 CREEK ENTERPRISES
            </b>
            <i>
                 (id: 6387)
            </i>
        </li>
        <li>
            <b>
                1M CUSTOM CAR TRANSPORTS, INC.
            </b>
            <i>
                 (id: 9172)
            </i>
        </li>
    </ul>
</div>

You should be able to read that code easily. The only new item is the `mapM_` function, which is a monadic iterator that doesn't return a result. You may wonder how the `li` elements are linked to the parent `ul` given the `mapM_` doesn't return anything (well, you should...).  That's an example of how the monadic types represent a context that can be modified. In this case, you can think of the ongoing invocation of Blaze functions as the ongoing building up of a context. That context is a virtual DOM that gets incrementally loaded with HTML elements.
Getting Monads is incredibly useful, but unfortunately it is not simple. You'll have to do some decent learning to go through the basics.  But you should, the concept is very powerful and will give you a much better understanding of computing in general.

But right now instead of digging into monads, we'll look into a very typical web app feature: the HTML template.

<h2>Using HTML templates (Jinja2)</h2>

In the Haskell ecosysteme, the Ginger package is the go-to for using Jinja2 templates.

In the following example, the `DotValue` structure will represent our app-specific information. Ginger logic is extended with meta-programming to `DotValue` for use in the template runtime context.  Then a text file is loaded (it's the template!), it is parsed and finally the Ginger VM runs the template with the data context to generate some HTML.

The first test template is using **Tailwind CSS** to add styling to the Jupyter notebook bland look, and it's also using the D3.js library for implementing data visualization. That has nothing to do with Haskell but just using *markdown* gets boring after a while.

First we've buit the `DotValue` type, which will hold 2 terminal values, the string and circle, and two container values, the list and the dictionary.  You can see here the common functional habit of using self-referencing definitions:

In [158]:
import Control.Monad.Identity (runIdentity)
import qualified Data.Map as Mp
import qualified Text.Ginger as Gi
import qualified Text.Ginger.Html as Gi

data DotValue =
  StrV T.Text
  | DictV (Mp.Map T.Text DotValue)
  | ListV [ DotValue ]
  | CircleV Int Int Int String  -- quick and dirty way to provide x,y pos, radius and color.

Then we provide the logic for transforming a `DotValue` value into a *Ginger*'s `GVal` one.  For the first 3 of the 4 kind of values, it's a simple remap already within Ginger default converter. The `CircleV` conversion is not handled by the default converter: it's a matter of mapping each of the record's fields into a named value, which is how the **Javascript** code in the template will know how to pass along the right data to the D3.js functions for getting **SVG** diagrams.

In [159]:
-- Enable the use of the "\case" shortcut (f x = case x of ... => f = \case ...):
{-# LANGUAGE LambdaCase #-}

instance Gi.ToGVal m DotValue where
  toGVal = \case
    StrV aString -> Gi.toGVal aString
    DictV aMap -> Gi.toGVal aMap
    ListV aList -> Gi.toGVal aList
    CircleV cX cY radius color -> Gi.toGVal $
            Mp.fromList [
              ("cx", show cX)  :: (T.Text, String)
              , ("cy", show cY)
              , ("radius", show radius)
              , ("color", color)
            ]

In this example we use a common Haskell structure, the tuple `(<value>, <value>)`.  Tuples are standard in most modern languages, and I expect they work just like you think they should.
The `Map` dictionary is initialized from a list of pairs (2-tuples); we giving typing information for the first element, and that is enough to get all the rest of the typing figured out.

Then we build some mock data and store it all in the `demoData_1` term.  Note that although we're dealing with complex typing possibilities, the compiler is still able to figure out what is what and we have no typing details for any function to provide.

Also let's recycle the data from the Vehicle API if there's some available.

In [160]:
demoNavList = case mbApiResponse of
  Nothing ->
    let
      nav_1 = Mp.fromList [
         ("url", StrV "https://gaga.com/url_1")
       , ("label", StrV "Gaga")
       ]
      nav_2 = Mp.fromList [
          ("url", StrV "https://gougou.com/url_1")
       , ("label", StrV "Gougou")
       ]
    in 
    ListV [ DictV nav_1, DictV nav_2 ]
  Just apiResponse ->
    let
      subList = Prelude.take 5 apiResponse.results
    in
      ListV $ Prelude.map carDescToTuple subList
  where
    carDescToTuple cd =
        DictV $ Mp.fromList [
            ("url", StrV . T.pack $ "http://https://vpic.nhtsa.dot.gov/api/vehicles/getmake?id=" <> show cd.uid)
            , ("label", StrV cd.name)
        ]

circles = [
    CircleV 20 90 40 "green"  
    , CircleV 50 90 45 "red"  
    , CircleV 85 90 50 "blue"  
  ]

demoData_1 = Mp.fromList [
    ("title", StrV "Haskell, Hidden Gem")
    , ("navigation", demoNavList)
    , ("circles", ListV circles)
  ]

As usual, no need to bother with type annotations. But it's worth looking under the hood and find out what was inferred for the `demoData_1` by using the `:t` feature of the notebook, as in:

In [161]:
:t demoData_1

demoData_1 :: forall {k}. (Ord k, IsString k) => Map k DotValue

This means the compiler can infer there's a `Map` of `DotValue` values involved (that's the dictionary and its values). Yet for the keys the only possible inference is on polymorphic constraints (`Ord k` means be ordered, and `IsString k` means must be able to be shown as a string). Even if the inference is only partial, it's enough to move forward. Furter use of that `demoData_1` function will resolve what kind of string variation the keys are.

While it may be convenient to let the compiler do all the type management, it is a good habit to lock in some of the types, so a more prudent approach of writing the `demoData` definition would be to specify:

    demoData_1 :: Mp.Map T.Text DotValue

if we want to make sure that the `Text` implementation of strings is used.

The `readFile` function takes care of loading data from a file, nothing special here.  But as this is an *impure* operation, we use the `<-` symbol to extract the result back into the *pure* context.
If the file doesn't exist, the `readFile` function will throw an exception (yes, Haskell uses that) and stop the execution of the logic.

In [162]:
template_1 <- Prelude.readFile "xstatic/templates/templ_1.html"

To handle included templates and errors that can occur during the parsing of the text just loaded up, the creation of the executable context is a bit more complex than just calling the `parseGinger' function:

In [163]:
loadTemplate aTemplate = do
  either (error . show) id . runIdentity $
    Gi.parseGinger (const $ return Nothing) Nothing aTemplate

The first parameter of `parseGinger` is a file resolver to handle included templates; it is a function that will receive a path and figure out how to fetch additional content. In our case, we pass a do-nothing function, `const $ return Nothing`.  Note that `return` is not a keyword, but a function that does the opposite of the `<-` extractor used in monadic context: it embeds a *pure* value into the monadic container. In the case of **IO** monad, it makes the `Nothing` become the equivalent of an IO operation that didn't produce anything.

The Ginger VM needs a way to dereference values during the template execution. The `lookup` function does that: it simply gets the variable name as parameter (`needle`) and finds that in the `demoData` dictionary.
Then the logic provided in the instance of `ToGVal` for `DotValue` will do the rest of the work in getting the end result into the constructed HTML.

In [164]:
derefLabel needle =
  Gi.toGVal $ Mp.lookup needle demoData_1

tmplDemo =
  Gi.runGinger (Gi.makeContextHtml derefLabel) (loadTemplate template_1)

Gi.htmlSource tmplDemo

Name,Destination
"1/OFF KUSTOMS, LLC","1/OFF KUSTOMS, LLC"
"102 IRONWORKS, INC.","102 IRONWORKS, INC."
12832429 CANADA INC.,12832429 CANADA INC.
17 CREEK ENTERPRISES,17 CREEK ENTERPRISES
"1M CUSTOM CAR TRANSPORTS, INC.","1M CUSTOM CAR TRANSPORTS, INC."


The `htmlSource` function simply converts the internal representation of HTML produced by the `runGinger` function into the one required by the notebook.

At this poing we've covered how to deal with JSON, fetch data from the web, create HTML dom from within Haskell code and using Jinja2 templates.

The next typical features a web app developer is working with are database operations and HTTP endpoint servicing. Just before moving forward, have a quick reflection on, if you want to do quick iteration on developing a good looking HTML page, how much simpler it is to use template-based HTML generation vs going through interpreted Blaze logic... You may find templates aren't that useful anymore...

<h2>Domain Specific Language and DB access</h2>

The Domain Specific Language (DSL) is a very cool feature of Haskell that is somewhat at odds with it.  Haskell is promoting simple syntax, a functional approach and such... and then suddenly with DSL the door opens to pretty much any syntax you can come up with.

DSLs syntax are brought in by doing the relevant package import and then using the *QuasiQuotes* blocks to switch between Haskell and a DSL syntax.  This also requires to enable the *QuasiQuotes* mode with:

In [165]:
{-# LANGUAGE QuasiQuotes #-}

We'll tackle DB access to showcase the benefits of DSLs. No matter how many ORM packages people come up with in their favorite ecosystem, you'll eventually conclude that SQL is the way to go when dealing with manipulations over large datasets. But normally that means you end up writing SQL statements as string constant with a bunch of question marks for parameter holes, invoke some exec functions, map out the results, etc.  It's error prone, it's very error prone, it's incredibly error prone and even awkward.  If only you could write SQL as a first-class citizen in the rest of the logic you can express with your favorite syntax.

The solution is *embedded SQL*, ie the ability to use your preferred language and insert SQL statements that will be analyzed, syntax-verified, type-impedence resolved, and that will be executed transparently at runtime.  And you want that in the latest version of the compiling toolchain at your disposition (Pro*C anyone?).

This is exactly what we can achieve with a SQL *DSL* in the GHC compiler, which means Haskell instantly becomes your favorite programming language.

The [Hasql-th](https://hackage.haskell.org/package/hasql-th) package provides the SQL syntax as a DSL as part of the excellent [Hasql](https://hackage.haskell.org/package/hasql) family of facilities for Postgresql operations. In other words, you get a great embedding of SQL in Haskell code.

Rather than doing function calls that pass SQL statements as string, we can instead do SQL queries that are syntax- and type-checked at compile time.  So much for code breaks at runtime because you wrote `slect a from ...` instead of `select a from ...`.

To get started, a bunch of packages are brought into the namespace.

In [166]:
import Data.ByteString

import qualified Hasql.Connection as DbC
import qualified Hasql.Session as Se
import Hasql.TH as Th

-- import GHC.Word (Word16)
-- import qualified Data.Time.Clock as Clk
import qualified Data.Vector as Vc

First, to connect, the proper way to get DB credentials should be to get them from *outside* the notebook, eg from the Jupyter process' environment. If there's a `DBINFO` env var that contains the connection params separated by ':' symbol, it would be a matter of spliting that var value into 4 entries and then use that to define the `dbSettings`:

In [167]:
import qualified System.Environment as SE
import qualified Data.Text.Encoding as T

dbInfo <- SE.getEnv "DBINFO"

[ host, db, user, paswd] = Prelude.map T.encodeUtf8 $ T.splitOn ":" (T.pack dbInfo)

**Note** that we get the env var values as a `String`, then convert that into a `Text` to use the `splitOn` function (`String` has no such feature) and get an array of values, and finally we convert each value to a `ByteString` with the `encodeUtf8`.  Always fun to do some string conversion gymnastics.

But to make things simpler we use a free-tier demo db from SupaBase, and thus hiding the connection information isn't worth it:

In [218]:
dbSettings = DbC.settings "db.dyqmhgvkpvltnfibeuyh.supabase.co" 5432 "demoa" "justForDemo" "demoha"

Our demo database provides basic information about the top starred and forked GitHub repos for the last 5 years that I've extracted and normalized from the CSV files compiled in [GitHub Top 100](https://github.com/EvanLi/Github-Ranking).

Within the Haskell execution context the `GhProject` structure will provide a way to hold the de-normalized data, and our DB access example will simply do a *select/join* over all normalized tables to create the data in very similarly to its original format.

In [219]:
import qualified Data.Time.LocalTime as Lt
import qualified Data.Int as DI

data GhProject = GhProject {
    rank :: DI.Int32
    , section :: T.Text
    , repo :: T.Text
    , stars :: DI.Int32
    , forks :: DI.Int32
    , language :: Maybe T.Text
    , url :: Maybe T.Text
    , uname :: T.Text
    , issues :: DI.Int32
    , lastCommit :: Lt.LocalTime
    , description :: Maybe T.Text
  }

data TopGithubCtxt = TopGithubCtxt {
    headColumns :: [ T.Text ]
    , rows :: GhProject
  }

Now we're getting to the real stuff: write a SQL statement to extract rows from the DB.

First, note that the syntax switch between Haskell and SQL is done using the `[<context>| <statements ]` *QuasiQuoter* block.

Second, we use the `[Th.vectorStatement|... |]` block to express that the SQL statement will return a vector of values. Then we provide the query using almost standard SQL; that code will go through a proper SQL parser to produce (internally) the code which will execute on the DB server.  Additionally the compiler will create glue code to pass values to the query from the Haskell world, and convert the raw data returned by the DB into a `Vector` of Haskell values.

To retrieve a bunch of rows from our GitHub dataset, we define the `simpleQuery` function with embedded SQL:

In [220]:
import qualified Data.Profunctor as Pf
import qualified Data.Tuple.Curry as Tc

simpleQuery repoName =
  Se.statement (repoName) $ Pf.dimap matcherFromString toGhProject
    [Th.vectorStatement|
        select
            o.rank::int4, ta.label::text, tb.label::text, o.stars::int4, o.forks::int4
            , tc.label::text?, td.label::text?
            , te.label::text, o.issues::int4, o.last_commit::timestamp, o.description::text?
        from GithubTop100 o
            left join itemu ta on o.itemfk = ta.id
            left join RepoNameU tb on o.reponamefk = tb.id
            left join LanguageU tc on o.languagefk = tc.id
            left join UrlU td on o.urlfk = td.id
            left join UserU te on o.unamefk = te.id
        where tb.label ilike $1::text
        order by date_trunc('day', o.last_commit) desc
            , itemfk, rank
        limit 10
    |]
  where
  matcherFromString n = "%" <> n <> "%"
  toGhProject = fmap (Tc.uncurryN GhProject)

By *almost standard SQL*, I meant that the standard SQL syntax is expanded with some type-annotations so that fields in the `select` statement are typed to both check consistency and specify SQL-to-Haskell value conversion.

The `statement` function's first argument specifies parameters to use as dynamic values in the SQL statement: in this case we pass `fromDate` to have an _earliest commit date_ filter for the results. The 2nd argument of the `statement` function can be either the *QuasiQuoter* block by itself or together with some conversion logic for parameters and results through a `dimap` association. Detailing the [ProFunctor](https://hackage.haskell.org/package/profunctors/docs/Data-Profunctor.html) package that provides `dimap` is beyond the scope of this notebook, but you can figure out that it uses an *encoder* function `dateFromString` which simply converts a string to a `Data.Time.LocalTime` value to be used in the `select` statement, and a *decoder* function `toGhProject` which simply converts raw tuples of values returned by the select into `GhProject` values through the application of the `uncurryN` helper (```uncurryN aFct (a1, a2, a3) = aFct a1 a2 a3```).

Parameters passed to the SQL block are positional, ie the `$1::<type>` notation is used.  Note that the implementator of *Hasql* preferred to use positional referencing and so this example uses this approach, but Haskell DSL supports named params in *QuasiQuoter* blocks and adding the [Hasql-Interpolate](https://hackage.haskell.org/package/hasql-interpolate/docs/Hasql-Interpolate.html) package will let you use `#{fromDate}` notation instead of `$1::<type>`. The Haskell-to-SQL conversion is automatic or uses the meta-programming when Haskell values are complex.

Now that there's a `simpleQuery` function with embedded SQL defined, the next thing is to execute it within a DB session to get a `Vector` of `GhProject` values (or an error).

Enough blah-blah, let's get a connection to the DB server:

In [221]:
dbConn <- DbC.acquire dbSettings

Going out over a socket to a DB server through internet is definitely not *pure*, so we use the `<-` to bring back the result from the `acquire` call in *pure* form.

The next step is to execute the query, and then continue based on the correct or broken SQL result.

It's actually now just a matter to invoke *Hasql*'s `run` function, but we wrap that call with the error management that will catch any anomaly.  It's always a very good approach to put a maximum of runtime checking around remote data operations.

Note that we're passing a string version of the filter date to `simpleQuery`; internally the `read` function will take care of converting a string value into the appropriate type, which in this case is `Data.Time.LocalTime`. But who cares about typing details, right?

And rather than just calling the `simpleQuery`, we use first principles and provide full error-checking:

In [222]:
dbVal <- case dbConn of
  Right conn -> do
    rezA <- Se.run (simpleQuery "bootcamp") conn
    case rezA of
      Left err -> pure . Left $ show err
      Right mbVal -> pure $ Right mbVal
  Left err -> pure . Left $ show err

At this point either the DB server executed the SQL statement and `dbVal` is a vector for `GhProject` values, or it is an error. The no-error/error situation is indicated with the [Either](https://hackage.haskell.org/package/base/docs/Data-Either.html) monadic type, which uses `Right` and `Left` as container meaning *no-error* and *error* respectively.

To look at the data received from the DB, we'll format using HTML. It's just nicer than a bland *print*...

At this stage we go the Jinja HTML template way and build a `GVal` instance of `GhProject` as a dictionnary of fields that will be used to populate the template.  Note that using *Generics* it's possible to do that just by derivation, but hopefully you find it more instructive to see a bunch of fields mapped to the *Ginger* template execution context.

The last line runs the template engine and the dataset received from the DB is shown (if available, the DB query may take a while to complete) in a HTML format:

In [229]:
import qualified Data.Maybe as Mb

template_2 <- Prelude.readFile "xstatic/templates/templ_5.html"


instance Gi.ToGVal m GhProject where
  toGVal p = Gi.toGVal $ Mp.fromList [
        ("rank", T.pack $ show p.rank) :: (T.Text, T.Text), ("section", p.section), ("repo", p.repo), ("stars", T.pack $ show p.stars)
        , ("forks", T.pack $ show p.forks), ("language", Mb.fromMaybe "<nil>" p.language), ("url", Mb.fromMaybe "<nil>" p.url)
        , ("uname", p.uname), ("issues", T.pack $ show p.issues), ("last_commit", T.pack $ show p.lastCommit)
        , ("description", Mb.fromMaybe "<nil>" p.description)
    ]

derefGhProj context needle =
  case needle of
    "columns" ->
      Gi.toGVal [
          "Rank" :: String, "Section", "Repo", "Stars", "Forks", "Language"
          , "Url", "User", "Issues", "Latest"
        ]
    "results" -> case context of
      Left _ -> Gi.toGVal ("error" :: T.Text)
      Right vcProjs -> Gi.toGVal $ Vc.toList vcProjs
    _ -> Gi.toGVal ("error" :: T.Text)


Gi.htmlSource $ Gi.runGinger (Gi.makeContextHtml (derefGhProj dbVal)) (loadTemplate template_2)

Rank,Section,Repo,Stars,Forks,Language,Url,User,Issues,Latest,Description
7,top-100-forks,Complete-Python-3-Bootcamp,25318,81840,Jupyter Notebook,Complete-Python-3-Bootcamp,Pierian-Data,120,2023-11-24 14:21:32,Course Files for Complete Python 3 Bootcamp Course on Udemy
7,top-100-forks,Complete-Python-3-Bootcamp,25304,81816,Jupyter Notebook,Complete-Python-3-Bootcamp,Pierian-Data,120,2023-11-24 14:21:32,Course Files for Complete Python 3 Bootcamp Course on Udemy
7,top-100-forks,Complete-Python-3-Bootcamp,25274,81761,Jupyter Notebook,Complete-Python-3-Bootcamp,Pierian-Data,119,2023-11-24 14:21:32,Course Files for Complete Python 3 Bootcamp Course on Udemy
7,top-100-forks,Complete-Python-3-Bootcamp,25276,81774,Jupyter Notebook,Complete-Python-3-Bootcamp,Pierian-Data,119,2023-11-24 14:21:32,Course Files for Complete Python 3 Bootcamp Course on Udemy
7,top-100-forks,Complete-Python-3-Bootcamp,25295,81802,Jupyter Notebook,Complete-Python-3-Bootcamp,Pierian-Data,120,2023-11-24 14:21:32,Course Files for Complete Python 3 Bootcamp Course on Udemy
7,top-100-forks,Complete-Python-3-Bootcamp,25280,81786,Jupyter Notebook,Complete-Python-3-Bootcamp,Pierian-Data,120,2023-11-24 14:21:32,Course Files for Complete Python 3 Bootcamp Course on Udemy
7,top-100-forks,Complete-Python-3-Bootcamp,25267,81756,Jupyter Notebook,Complete-Python-3-Bootcamp,Pierian-Data,119,2023-11-24 14:21:32,Course Files for Complete Python 3 Bootcamp Course on Udemy
7,top-100-forks,Complete-Python-3-Bootcamp,25328,81847,Jupyter Notebook,Complete-Python-3-Bootcamp,Pierian-Data,120,2023-11-24 14:21:32,Course Files for Complete Python 3 Bootcamp Course on Udemy
7,top-100-forks,Complete-Python-3-Bootcamp,25313,81832,Jupyter Notebook,Complete-Python-3-Bootcamp,Pierian-Data,120,2023-11-24 14:21:32,Course Files for Complete Python 3 Bootcamp Course on Udemy
7,top-100-forks,Complete-Python-3-Bootcamp,25333,81862,Jupyter Notebook,Complete-Python-3-Bootcamp,Pierian-Data,120,2023-11-24 14:21:32,Course Files for Complete Python 3 Bootcamp Course on Udemy


That's it, we can interact with a SQL server in a safe and efficient way, it's easy to convert data back and forth between Haskell structures and DB tables, and it's also easy to create HTML out of datasets from Jinja templates.

For fun, let's look at another DSL: Javascript.
Yep, it's possible to write JS code as part of the Haskell code, have it syntax-checked and then sent to a **Node.js** engine for processing.

In [174]:
{-# LANGUAGE DerivingVia #-}
{-# LANGUAGE TemplateHaskell #-}
{-# LANGUAGE TypeApplications #-}
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE DeriveAnyClass #-}
{-# LANGUAGE ViewPatterns #-}

In [175]:
import Control.Exception (try, SomeException, evaluate)
import Control.Concurrent (threadDelay)

import Data.Aeson ( FromJSON )
import GHC.Generics ( Generic )
import Language.JavaScript.Inline
import Language.JavaScript.Inline.Core
import qualified Data.ByteString.Lazy as LBS
import Data.Text (Text, unpack)

import Control.Monad (ap)
import GHC.TypeLits (KnownSymbol)
import Data.Text.Array (run)
import Data.Vector.Generic.Mutable (set)

In [176]:
newtype JSReturn = JSReturn {
    result :: String
  } 
  deriving (Generic, Show, FromJSON)
  deriving FromJS via (Aeson JSReturn)

In [177]:
runJSTest :: IO JSReturn
runJSTest = do
  session <- newSession defaultConfig
  putStrLn "@[runJSTest] starting."
  rezA <- eval session [js|
    console.warn("@[runJSTest] JS starting.")
    let a = "allo"
    let b = "byebye"

    doTest = async () => {
      console.warn("@[runElmTest] JS doTest start.")
      setTimeout(() => console.warn("timeout!"), 5 * 1000)
      console.warn("@[runElmTest] JS doTest end.")
      return { "result": "run_JS_1: " + a + ", " + b}
    }
    finalRez = await doTest()
    console.warn("@[runJSTest] JS ending.")
    return finalRez
   |]
  closeSession session
  putStrLn "@[runJSTest] finishing."
  pure rezA

In [178]:
runJSTest

@[runJSTest] starting.
@[runJSTest] JS starting.
@[newSession] msg : JSEvalRe@[runElmTest] JS doTest start.
sponse {js@[runElmTest] JS doTest end.
@[runJSTest] JS ending.
EvalResponseId = 2868, jsEvalResponseContent = Right "{\"result\":\"run_JS_1: allo, byebye\"}"}
timeout!
@[runJSTest] finishing.
JSReturn {result = "run_JS_1: allo, byebye"}

In [179]:
runJS_B :: IO JSReturn
runJS_B = do
  session <- newSession defaultConfig
  rezA <- eval session [js|
    console.warn("@[runJSTest] JS starting.")

    console.warn("@[runJSTest] JS ending.")
    return { "result": "JS_B: pwd = " + process.cwd() }
   |]
  closeSession session
  pure rezA

In [180]:
runJS_B

@[runJSTest] JS starting.
@[newSession] msg : JSEvalResponse {jsEvalResponseId = 2928, @[runJSTest] JS ending.
jsEvalResponseContent = Right "{\"result\":\"JS_B: pwd = /home/lhugo/Projets/Fudd/IHaskell/Notebooks\"}"}
JSReturn {result = "JS_B: pwd = /home/lhugo/Projets/Fudd/IHaskell/Notebooks"}

<h2>Serving some Web endpoints: the very basics</h2>

The Haskell ecosystem has defined a long time ago a standard Web Application Interface (WAI) for building up web services. The [Wai](https://hackage.haskell.org/package/wai) package provides the basic layer for handling and responding to HTTP requests.

Then on top of that came a fast, light-weight web server as the [Warp](https://hackage.haskell.org/package/warp) package. Together WAI and Warp provide a simple way to prototype very quickly a web service.

First let's use the WAI/Warp pair to build a quick-and-dirty web handler, as one does for quick prototyping of an idea while testing stuff in the interpreter.

As usual, a few packages need to be brought into the namespace to move forward.

In [181]:
import qualified Control.Concurrent as Cc
import qualified Network.Wai as W
import qualified Network.Wai.Handler.Warp as Wr
import qualified Network.HTTP.Types as Ht

Then we create a basic HTTP handler called `waiDemoServer`, which we'll *hook* in the _marco_ path of the requesting URL, and that we'll use as a unconditional responder of the Jinja template content we loaded some time ago.

In [182]:
waiDemoServer request sendFct =
  sendFct $ W.responseLBS
    Ht.status200
    (case Prelude.lookup "marco" $ W.requestHeaders request of
       Nothing -> []
       Just val -> [("Polo", val)])
    (LBS.fromStrict . T.encodeUtf8 . Gi.htmlSource $ tmplDemo)

The arguments for `waiDemoServer` are the request received from the HTTP client and a responder function; that is standard stuff defined by WAI.

The behavior will be to reply with a 200 (OK) status code, and if the /marco path has been requested to send a `Polo` value in the headers of the response, together with the main body which is our filled template.

The service is started with a `run <port> <service-function>`, that's it.  In this example, we'll fork the service to a sub-process that we can kill later:

In [183]:
tid <- Cc.forkIO (Wr.run 3001 waiDemoServer)
print tid
:! date

ThreadId 10843

Tue Dec  5 11:05:36 PM +04 2023

As this may end up running in a private port on a virtual server, the following function can be used to do a HTTP GET on the service and visualize the HTML from the response (here we're simply reusing the basic HTTP client introduced previously).

In [184]:
fetchDemo =
  let
    url = "GET http://localhost:3001/marco"
    request = N.setQueryString [("arg", Just "123")] $ N.parseRequest_ url
  in do
    reqMgr <- N.newManager N.defaultManagerSettings
    N.httpLbs request reqMgr

reply <- fetchDemo
ID.html . TL.unpack . TL.decodeUtf8 $ reply.responseBody

Name,Destination
"1/OFF KUSTOMS, LLC","1/OFF KUSTOMS, LLC"
"102 IRONWORKS, INC.","102 IRONWORKS, INC."
12832429 CANADA INC.,12832429 CANADA INC.
17 CREEK ENTERPRISES,17 CREEK ENTERPRISES
"1M CUSTOM CAR TRANSPORTS, INC.","1M CUSTOM CAR TRANSPORTS, INC."


When you have done the request (and hopefully you can actually do a HTTP GET from your device directly), kill the background process to keep the execution space clean. By the way I've noticed there's a bug in Jupyter and sometimes the SVG from the page fetched doesn't show up... Anyways, you still get the idea whether it does or not.

In [185]:
Cc.killThread tid
:! date

Tue Dec  5 11:05:37 PM +04 2023

<h2>Our objective: creating production grade modern web apps</h2>

This is finally the conclusion of this whole notebook: being able to produce a _normal, modern, efficient_ web app, based on HTMX and Tailwind in the frontend, and with server-side rendering of templates filled with data provided by SQL queries and with websocket low-latency communications. That plus authentication, security, distributed computing, 3rd party APIs, Javascript/C++/Python ecosystems integration, ie the usual stuff.

You've already seen how to deal with JSON, how to render HTML from either server-side DOM (Blaze) or Jinja templates (Ginger), how to query a Postgresql database using embedded SQL, and how to do HTTP operations (client & handler).

And hopefully you have reached this cell in a _short_ amount of time.

To do the final part, you'll finally encounter the power of the Haskell type system. This is because the wrap-up of everthing covered so far will happen using the [Servant](https://hackage.haskell.org/package/servant) package, a powerful web app server based on type combinators.

The core idea of Servant is to enable us to use type equations to specify the API of our web service. By doing so, all kind of logic can be applied to automatically deduce the way endpoint routes are managed, how security is applied, how middleware modify the request & reply behaviors, how client-side code can be generated, how compliance code can be produced, etc.

In order to implement all the functionality in the notebook, we'll have to deal with a few limitations of the environment.  For some reason, I can't get the local module import to work if the module hasn't been implemented within the notebook, so you'll see the `WebServer.CorsPolicy` and `WebServer.JWT` modules in the coming cells, although that kind of helper logic should just be imported from local files.

I put the language mode settings, the long list of imported packages and the 2 modules as collapsed cells.  You can have a look at the code if you want to, but it's really not necessary for getting in the main part of the implementation.

Then to document the code with *markdown* cells I also had to define every symbol (function, type) before it is used.  That ended up putting the code in the opposite way I'd normally write it, eg I'd normally go top-down and write the main logic (`listen`) first then define more and more of the details used by the main logic. Hopefully it's an order that you can live with. Probably that some blocks will seem pretty abstract when they show up, and you'll have to wait for the code that uses what they define to better understand how the Servant system works.

In [186]:
{-# LANGUAGE DataKinds #-}
{-# LANGUAGE TypeApplications #-}
{-# LANGUAGE TypeOperators #-}
{-# LANGUAGE TypeFamilies #-}
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE DerivingStrategies #-}
{-# LANGUAGE DeriveAnyClass #-}
{-# LANGUAGE GeneralizedNewtypeDeriving #-}

In [187]:
import Control.Concurrent.Async (concurrently_)
import Control.Exception (bracket)
import Control.Monad (forever, void)
import Control.Monad.Catch (MonadCatch, MonadMask, MonadThrow)
import Control.Monad.Cont (runContT, ContT (..), Cont)
import Control.Monad.Except (ExceptT, MonadError, withExceptT)
import Control.Monad.IO.Class (MonadIO, liftIO)
import Control.Monad.Reader (ReaderT, MonadReader, runReaderT, asks)
import Control.Exception.Safe (tryAny)

import Data.List.NonEmpty (NonEmpty (..))
import qualified Data.Int as DI

import Data.Aeson (FromJSON (..), ToJSON (..), Value (Object), (.:), (.:?), eitherDecode)

import Network.Wai.Handler.Warp as Wr
import Network.Wai.Parse (setMaxRequestKeyLength, defaultParseRequestBodyOptions)
import Network.HTTP.Media ((//), (/:))
import Network.Wai.Middleware.RequestLogger (logStdout)
import Network.Wai.Middleware.Servant.Errors (errorMw)
import qualified Network.WebSockets as Ws

import GHC.Stack (HasCallStack)
import Servant as Srv
import Servant.API.Generic
import Servant.API.ContentTypes (FormUrlEncoded)
import Servant.Auth.Server (Auth, AuthResult (..), BasicAuth, BasicAuthCfg, CookieSettings (CookieSettings, cookieIsSecure)
                  , IsSecure (NotSecure), FromBasicAuthData, JWT, JWTSettings, FromJWT (..), ToJWT (..), cookieIsSecure
                  , defaultCookieSettings, defaultJWTSettings )
import qualified Servant.Auth.Server as Sauth
import Servant.Multipart (defaultMultipartOptions, MultipartOptions (..), Tmp)
import Servant.Server.Generic (AsServerT, genericServerT)
import Servant.API.WebSocket (WebSocket)
import Web.FormUrlEncoded (FromForm (..))

import System.Posix.Signals as Sgnl

import qualified Text.Blaze.Htmx as A
import qualified Text.Blaze.Html.Renderer.Utf8 as H
import qualified Text.Blaze.Htmx as X
import qualified Text.Blaze.Htmx.WebSockets as X


In a normal coding environment, the helper logic would just be brought into a the main definintions, like this:

In [188]:
-- Bring in the Cors Policy & JWT helpers to configure the HTTP handler.

import WebServer.CorsPolicy (setCorsPolicy)
import WebServer.JWT (generateKeyPairIO, readJWK)

import qualified Options.Runtime as Ropt

Here are some structures and logic extensions that we'll use throughout the endpoint handlers.

The `AppEnv` structure will be used to represent a global state for the web server, and hold:

- the JWT settings used most importantly to handle the encoding/decoding of JW tokens in HTTP requests
- the global DB context to pass to handlers so they can do SQL operations,
- the execution options that control the setup of the web server (we'll only use a port value).

In [189]:
data AppEnv = AppEnv {
    jwtSettings :: JWTSettings
  , rtOptions :: Ropt.RunOptions
  , dbCtxt :: Maybe DbC.Connection
  }

The `HTML` type is a place holder to tag the result of an endpoint handler with a MIME type (text/html) to be returned to the requesting client.
The `RawHtml` is a demo of how to pass results back up the responder chain; in this case we'll only use it to hold some textual data, eg a long string that is a HTML page that needs to be sent back to the client.  But in real-life you could see a more complex kind of definition where the `instance MimeRender HTML RawHtml` logic extension could do all kind of post-processing to compute the data actually sent back to the client.

In [190]:
data HTML = HTML
newtype RawHtml = RawHtml { rawContent :: LBS.ByteString }

instance Accept HTML where
  contentType _ = "text" // "html" /: ("charset", "utf-8")

instance MimeRender HTML RawHtml where
  mimeRender _ = rawContent

By the way instead of defining the `RawHtml` type with the `data` keyword, here the alternative `newtype` keyword is used instead. A `newtype` still creates a type, but it's a more constraint kind of structure with less runtime overhead; for example, it contains only one value in a record.  Using `data` would also work fine but doing so gets the compiler to complain that a `newtype` is better...

To handle secure data exchange between client and server we pass a JWT, which will contain securely the `SessionContext` structure. The conversion from/to JSON and the encryption/decryption are implemented by doing a derivation of the `FromJSON`, `ToJSON`, `FromJWT` and `ToJWT` classes (which aren't OO classes!), which automagically makes everything work. 

In [191]:
newtype SessionContext = SessionContext {
    sessionID :: DI.Int32
  }
  deriving stock Generic
  deriving anyclass (FromJSON, ToJSON, FromJWT, ToJWT)

To authenticate itself, the client will send a simple JSON block with a `identity` and `trust` values, which the `LoginForm` structure provides. If the authentication succeeds, the client will receive a JSON block with a context and an encrypted JWT, which is provided by the `LoginResult` structure.  Same thing here, the `FromJSON` and `ToJSON` derivation take core of the conversion process automagically.

In [192]:
data LoginForm = LoginForm {
  identity :: T.Text
  , trust :: T.Text
  }
  deriving stock (Eq, Show, Generic)
  deriving anyclass (ToJSON, FromJSON)

data LoginResult = LoginResult {
    context :: SessionContext
    , jwt :: T.Text
  }
  deriving stock Generic
  deriving anyclass (ToJSON)

One endpoint provides a search feature.  The client will use a *HTML* `form` element to implement the capture and transmission of the needle to find; the `SearchContent` structure represents that and it is derived with `FromForm` to trigger automatically the conversion from the HTTP request *application/x-www-form-urlencoded* data into a Haskell `Text` value. 

In [193]:
newtype SearchContent = SearchContent {
    needle :: T.Text
  }
  deriving stock (Eq, Show, Generic)
  deriving anyclass (FromForm)

We finally get to an important part of the web server implementation: the route definitions. Using Servant combinators, the routes are explained as a set of structure definitions, the top-level one being `ServerRoutes`.

Servant routing specification is great for large-scale systems. To demo that point, we split the routing at the top-level into two branches: the `anonymous` and `authenticated` ones.  Nothing special happens for the `anonymous` branch and the definition just continues at the `AnonymousRoutes` type, but you can already see some processing specification that happens for the `authenticated` branch.

The

 ```route :- Auth '[JWT, Sauth.BasicAuth] SessionContext :> ToServantApi AuthenticatedRoutes```

type equation means that first the request handling logic will extract authentication and session information before it is passed to the next level, defined in the `AuthenticatedRoutes` type. The `'[...]`, `:-` and `:>` symbols are functions, and further more they happen to be type-level functions (the Servant **combinators**), ie they operate with types rather than values (for the heavy-duty functional folks: that's 2nd order logic).

The end-result of Servant advanced type operations is that we can name routes easily, we can group routes in logical groups, we can assemble route definitions, and we can specify formally the data structures that will come in and out of the HTTP request processing and handling. For the API that grows, this leads to serious time saving during refactoring, cleaning and testing.

In [194]:
data ServerRoutes route = ServerRoutes {
    anonymous :: route :- ToServantApi AnonymousRoutes
    , authenticated :: route :- Auth '[JWT, Sauth.BasicAuth] SessionContext :> ToServantApi AuthenticatedRoutes
  }
  deriving (Generic)


data AnonymousRoutes route = AnonymousRoutes { 
    login :: route :- "inlogin" :> ReqBody '[JSON] LoginForm :> Post '[JSON] LoginResult
    , staticLink :: route :- "xstatic" :> CaptureAll "path" String :> Get '[HTML] RawHtml
    , stream :: route :- "stream" :> WebSocket
    , search :: route :- "xsearch" :> ReqBody '[FormUrlEncoded] SearchContent :> Post '[HTML] RawHtml
    , homePage :: route :- Get '[HTML] RawHtml
  }
  deriving (Generic)


newtype AuthenticatedRoutes route = AuthenticatedRoutes { 
    privateAccess :: route :- "private" :> ToServantApi PrivateRoutes
  }
  deriving (Generic)


newtype PrivateRoutes route = PrivateRoutes { 
    getPage :: route :- "static" :> Capture "path" String :> Get '[HTML] RawHtml
  }
  deriving (Generic)

The `AnonymousRoutes` then specify 5 routes; you can probably guess what is going already if you read those 5 definitions:

- the `login` route is at _/inlogin_ (first part), it expects a JSON block from the client in the form of `LoginForm` (`ReqBody ...`), it comes in as a POST request and the handler will provide a `LoginResult` that will sent back as a JSON block (`Post ...`).  That's the route a client uses to authenticate itself.
- the `staticLink` route is at _/xstatic_, it expects more segments in the URL and that will be globbed up as a list of strings (`CaptureAll ...`; the `path` identifier is just a place-holder), it comes as a GET request and the handler will return some long text that the client will get as *text/html* content (` '[HTML] RawHtml`).
- the `stream` route is at _/stream_, and it's going to operate as a websocket connection.
- the `search` route is at _/xsearch_, it expects a *urlencoded* form in the URL in the form of `SearchContent`, it comes in as a POST request and the handler will return some long text that the client (it's starting to be repetitive).
- the `homePage` route is _/_, it simply comes in as GET request and returns *text/html*, ie a the HTML for the home page.

The `AuthenticatedRoutes is simply a grouping that prefixes the authenticated routes to _/private_ and forwards to `PrivateRoutes` definitions, again some overhead just to show the convenient structuring of route definitions in Servant.

Finally the `PrivatesRoutes` defines a `getPage` route located at _/private/static_, which will glob the rest of the URL, comes in as a GET and for which the handler will return a HTML page, just like the anonymous static handler.

That will be it for the routes of our web service.  Now that these are specified as types, the entire conformance and meta-programming power of Haskell is available to analyse the specifications, and generate all kind of further results.  There's no extra description file required, no pre-processing or boiler-plate code generation required.

Also the meta-programming power of Haskell is available to create these routes from other specifications, for example the [Morpheus](https://hackage.haskell.org/package/morpheus-graphql) package provide you with the ability to read a GraphQL spec file during compilation and deduce the entire API within the compilation pass, ie there's no boiler-plate code or temp files to deal with. No pre-compilation, no transpilation, no post-tweaking.

The `DemoServiceError` structure provides a specific error management for our handlers, ie during processing the handlers can throw these kind of errors and the Servant logic will glue them back into a response for the client's request.  The `asHttpError` function does the conversion from our web service error system to standard HTTP results.

In [195]:
data DemoServiceError
  = NotImplemented
  | UnexpectedError T.Text
  | NotAuthorized T.Text
  | Unaccessible
  | NotFound T.Text
  deriving stock (Generic, Show, Eq)
  deriving anyclass (ToJSON, FromJSON)


asHttpError :: DemoServiceError -> ServerError
asHttpError err =
  case err of
    Unaccessible -> err401 { errBody = "Resource not accessible." }
    NotImplemented -> err500 { errBody = "Not Implemented." }
    UnexpectedError x -> err500 { errBody = textToLBS x }
    NotAuthorized x -> err401 { errBody = textToLBS x }
    NotFound x -> err404 { errBody = textToLBS x }
  where
    textToLBS = LBS.fromStrict . T.encodeUtf8

Finally we define the *glue* type `WebApp` that will give an execution context to the all the handlers (monad stuff) in Servant main control logic. The important thing here is to specify the global data context `AppEnv` and the error management we use, `DemoServiceError`. Note that the `WebApp` type is parametrized as the return structure used by each route handler is specific to the route.

In [196]:
newtype WebApp a = WebApp { 
    runApp :: ReaderT AppEnv (ExceptT DemoServiceError IO) a
  }
  deriving newtype (
    Functor, Applicative, Monad, MonadMask, MonadCatch, MonadThrow
    , MonadReader AppEnv, MonadIO, MonadError DemoServiceError
  )

Then comes a first route handler, `loginHandler`, which will get a LoginForm as a parameter and will return a LoginResult. Remembering the route definintions, this will be attached a bit further down in the *handler associations* section to take care of the _/inlogin_ endpoint (I definitely prefer a top-down than bottom-up approach).

As this is a demo without a real authentification system, the `loginHandler` simply pulls out JWT stuff with:

   ```asks jwtSettings```

which extracts the info from the `jwtSettings` field in the `AppEnv` value that represents the global runtime context (here's part of Haskell monad magic at work). If there's no problem dong so, it builds a new encrypted token with a fake `SessionContext` and puts that into a `LoginResult` value that is the result of the `loginHandler` call.  If there's a problem, an `UnexpectedError` is thrown.

In a production environment, the authentication system would be provided in the `AppEnv` value, and it would be used to assess the validity of the login data; or alternatively a 3rd-party authentication system would have already provided a JWT for the client and it would simply be verified by loginHandler, ie there would be no `LoginForm` at play and instead the route definition would specify how the JWT shows up (in headers, as a URL parameter, etc).

In [197]:
loginHandler :: LoginForm -> WebApp LoginResult
loginHandler form = do
  settings <- asks jwtSettings
  jwtRez <- liftIO $ Sauth.makeJWT (SessionContext 1) settings Nothing
  case jwtRez of
    Left err ->
      throwError . UnexpectedError . T.pack $ show err
    Right jwtValue ->
      pure $ LoginResult {
          context = SessionContext 1
          , jwt = T.decodeUtf8 . LBS.toStrict $ jwtValue
        }

Our `homePage` handler is picking up a Jinja template, putting some values in its render context from the DB without any caching or pooling to keep it simple, and sends that back as the result to the client.

In [198]:
derefLabelB needle =
  Gi.toGVal $ Mp.lookup needle demoData_1

loadTemplateB aTemplate = do
  either (error . show) id . runIdentity $
    Gi.parseGinger (const $ return Nothing) Nothing aTemplate

homePageHandler :: WebApp RawHtml
homePageHandler = do
  template <- liftIO $ Prelude.readFile "xstatic/templates/templ_4.html"
  let
    genHtml = Gi.htmlSource $ Gi.runGinger (Gi.makeContextHtml derefLabelB) (loadTemplateB template)
  pure . RawHtml . LBS.fromStrict . T.encodeUtf8  $ genHtml


The `searchHandler` is intended to respond to a HTMX request, so it will reply with HTML-formatted content where the initial values are fetched from the DB:

In [239]:
searchHandler :: SearchContent -> WebApp RawHtml
searchHandler searchContent = do
  mbDbCtxt <- asks dbCtxt
  dbVal <- case mbDbCtxt of
    Just conn -> do
      rezA <- liftIO $ Se.run (simpleQuery searchContent.needle) conn
      case rezA of
        Left err -> pure . Left $ show err
        Right mbVal -> pure $ Right mbVal
    Nothing -> throwError . UnexpectedError $ "No database available."
  let
    genHtml = Gi.htmlSource $ Gi.runGinger (Gi.makeContextHtml (derefGhProj dbVal)) (loadTemplate template_2)
  pure . RawHtml . LBS.fromStrict . T.encodeUtf8  $ genHtml

The `anonStaticHandler` simply loads up files from the *xstatic/assets* local notebook directory and sends that data back to the client as a *text/html* reply.

In [201]:
anonStaticHandler :: [ String ] -> WebApp RawHtml
anonStaticHandler pageUrls = do
  let
    fullPath = Prelude.foldl (\accum segment -> accum <> "/" <> segment) "xstatic" pageUrls
  pageContent <- liftIO $ LBS.readFile fullPath
  pure . RawHtml $ pageContent

The `prvStaticHandler` does the same job as the `anonStaticHandler`, but it gets authentication and session information so it can allow/block or modify content access based on that. But for the sake of simplicity in this demo it's not dealing with that information.

In [202]:
prvStaticHandler :: AuthResult SessionContext -> String -> WebApp RawHtml
prvStaticHandler authResult pageUrl =
  case authResult of
    Authenticated context ->
      -- TODO: fetch an interesting piece of information in HTML.
      let
        content = H.renderHtml $ H.div $ H.toHtml $ T.pack . show $ context.sessionID
      in
        pure $ RawHtml content
    _ ->
      throwError . NotAuthorized . T.pack $ pageUrl

Finally we have the `streamHandler`, which will receive to requests over a websocket connection with a client and will reply on the same connection with HTML-formatted data, as it is intented to be use by a HTMX-enabled element.

In [203]:
streamHandler :: MonadIO m => Ws.Connection -> m ()
streamHandler conn = do
  liftIO $ Ws.withPingThread conn 30 (pure ()) $ do
    -- liftIO $ Ws.sendTextData conn ("<div id=\"notifications\" hx-swap-oob=\"beforeend\">Some message</div?" :: ByteString)
    handleClient
  where
    handleClient = do
      rezA <- tryAny $ forever receiveStream
      case rezA of
        Left err -> do
          liftIO . putStrLn $ "@[streamHandler] situation: " <> show err
          closeConnection
        Right _ -> do
          liftIO $ putStrLn "@[streamHandler] client disconnected."
          pure ()

    receiveStream = do
      rezA <- Ws.receiveDataMessage conn
      case rezA of
        Ws.Text msg decodedMsg ->
          let
            hxMsg = eitherDecode msg :: Either String HxWsMessage
          in
          case hxMsg of
            Left err -> do
              putStrLn $ "@[receiveStream] invalid HxWsMessage: " <> (T.unpack . T.decodeUtf8 . LBS.toStrict) msg
              putStrLn $ "@[receiveStream] error: " <> show err
            Right hxMsg ->
              Ws.sendTextData conn $ H.renderHtml $ htmxReply hxMsg.wsMessage
        Ws.Binary msg ->
          putStrLn "@[receiveStream] received binary."
    
    closeConnection = do
      Ws.sendClose conn ("Bye" :: ByteString)
      void $ Ws.receiveDataMessage conn

    htmxReply aMessage =
      H.tbody H.! A.id "notifications" H.! X.hxSwapOob "beforeend" $ do
        H.tr $ do
          H.td H.! A.class_ "px-6 py-4 whitespace-nowrap text-sm text-slate-900" $ H.toHtml aMessage
                

data HxWsHeaders = HxWsHeaders {
    request :: T.Text
    , trigger :: T.Text
    , triggerName :: Maybe T.Text
    , target :: T.Text
    , currentURL :: T.Text
  }
  deriving stock (Show, Generic)

instance FromJSON HxWsHeaders where
  parseJSON (Object obj) = HxWsHeaders <$>
    obj .: "HX-Request"
    <*> obj .: "HX-Trigger"
    <*> obj .:? "HX-Trigger-Name"
    <*> obj .: "HX-Target"
    <*> obj .: "HX-Current-URL"


data HxWsMessage = HxWsMessage {
    wsMessage :: T.Text
    , headers :: HxWsHeaders
  }
  deriving (Show, Generic)


instance FromJSON HxWsMessage where
  parseJSON (Object obj) = HxWsMessage <$>
    obj .: "ws-message"
    <*> obj .: "HEADERS"

The HTMX logic on the client is using a JSON block to encapsulate the request from the client, so we build the `hxWsMessage` and `HxWsHeaders` structures to take care of that, extending the `FromJSON` logic to match the format used by HTMX. In this case, the `instance FromJSON HxWsHeaders` function isn't as simple as for example the **Go** annotation, but if one does a bit of research there's a camel-case formating function in some Haskell package that can implement automatic field name rewriting and that would be more efficient to use.

We're almost done! After having defined the routes and handlers, we connect them together in the next block of code by specifying functions that use Servant's `genericServerT` connector and creating a value for each of the route types we've created a few cells ago and associating the handler function that implement the endpoint.

The ultimate glue point for all these specifications to Servant general request handling logic is the weird looking

    serverApiProxy :: Proxy (ToServantApi ServerRoutes)
    serverApiProxy = Proxy

It's a typing trick of Haskell, where the important part is the type definition `Proxy (ToServantApi ServerRoutes)` for the function `serverApiProxy`.  That will be analyzed by the compiler to figure out all kind of futher logic expansion using the definitions of `ServerRoutes`. The actual implementation is just providing a generic value of that type, using the constant `Proxy` value constructor, and has no real use.

In [240]:
-- Handler associations:
serverApiProxy :: Proxy (ToServantApi ServerRoutes)
serverApiProxy = Proxy


serverApiT :: ToServant ServerRoutes (AsServerT WebApp)
serverApiT =
  genericServerT $ ServerRoutes {
    anonymous = anonHandlers
    , authenticated = authHandlers
  }


anonHandlers :: ToServant AnonymousRoutes (AsServerT WebApp)
anonHandlers =
  genericServerT $ AnonymousRoutes {
    login = loginHandler
    , staticLink = anonStaticHandler
    , stream = streamHandler
    , homePage = homePageHandler
    , search = searchHandler
  }


authHandlers :: AuthResult SessionContext -> ToServant AuthenticatedRoutes (AsServerT WebApp)
authHandlers authResult =
  genericServerT $ AuthenticatedRoutes {
    privateAccess = prvStaticHandler authResult
  }

As the web service uses *form/url-encoded* authentication, there's a logic extension provided for our `SessionContext` structure which could do some validation and book-keeping, but in this demo, it will just pass on the logic to the default validation function supplied, `authCheckFun`.

In [205]:
type instance BasicAuthCfg = Srv.BasicAuthData -> IO (AuthResult SessionContext)
instance FromBasicAuthData SessionContext where
  fromBasicAuthData authData authCheckFun = authCheckFun authData

We have reached the operation side of running a web service! First, the WAI control is defined, where the listen port, the pre-processing logic (`showBanner`!) and the process interruption management are defined.

In [206]:
setupWai :: Int -> IO () -> Settings
setupWai port shutdownCallback =
  Wr.setPort port . Wr.setGracefulShutdownTimeout (Just 5) . Wr.setInstallShutdownHandler shutdownHandler
    . setBeforeMainLoop showBanner
    $ Wr.defaultSettings
  where
    showBanner =
      putStrLn $ "@[setupWai] using port: " <> show port
    shutdownHandler closeSocket = do
      void $ installHandler Sgnl.sigTERM (Catch $ shutdownCallback >> closeSocket) Nothing
      void $ installHandler Sgnl.sigINT (Catch $ shutdownCallback >> closeSocket) Nothing
      void $ installHandler Sgnl.sigQUIT (Catch $ shutdownCallback >> closeSocket) Nothing
      void $ installHandler Sgnl.sigHUP (Catch $ shutdownCallback >> closeSocket) Nothing

Then the `runAPI` is the overall bootstrap logic for the servicing. It takes care of setting up the JWT encryption components, the DB connection, it puts the middlewares in place (logging, multi-part requests handler, CORS policy handlers), creates the global context `appEnv` that each handler will work with, 

In [241]:
runAPI ::  Ropt.RunOptions -> IO Application
runAPI rtOpts = do
  -- Initialize the JWT parameters:
  myKey <- case rtOpts.jwkConfFile of
    Nothing ->
      generateKeyPairIO "/tmp/jwk.json"
    Just aPath ->
      readJWK aPath

  -- Initialize the DB parameters:
  eiConn <- DbC.acquire $ DbC.settings "db.dyqmhgvkpvltnfibeuyh.supabase.co" 5432 "demoa" "justForDemo" "demoha"
  case eiConn of
    Left err ->
      putStrLn $ "@[runAPI] db connection err: " <> show err
    Right _ -> pure ()

  -- pure computations:
  let
    cookieCfg = defaultCookieSettings { cookieIsSecure = NotSecure }
    jwtDefSettings  = Sauth.defaultJWTSettings myKey
    -- For file upload support, will be used in next demo:
    multipartOpts = (defaultMultipartOptions (Proxy :: Proxy Tmp)) { 
          generalOptions = setMaxRequestKeyLength 512 defaultParseRequestBodyOptions
      }

    -- Define the overall Servant pre-processing required on requests:
    runContext = cookieCfg :. jwtDefSettings :. fakeSessionValidation :. multipartOpts :. EmptyContext
    runCtxtProxy = Proxy :: Proxy '[CookieSettings, JWTSettings, BasicAuthCfg]

    -- activate some middlewares for loggin, CORS, standardized error handling:
    middlewares = linkUp $ id :| case rtOpts.corsPolicy of
      Nothing -> [ logStdout, errorMw @JSON @'["message", "status" ] ]
      Just aPolicy -> [ logStdout, setCorsPolicy aPolicy, errorMw @JSON @'["message", "status" ] ]

    mbDbConn = case eiConn of Left _ -> Nothing ; Right aConn -> Just aConn
    -- define the global runtime context for handlers:
    appEnv = AppEnv { jwtSettings = jwtDefSettings, rtOptions = rtOpts, dbCtxt = mbDbConn }
    -- link all that stuff in Servant magic:
    server = hoistServerWithContext serverApiProxy runCtxtProxy (toHandler appEnv) serverApiT

  -- launch the servicing:
  pure $ middlewares $ serveWithContext serverApiProxy runContext server
  where
    linkUp :: NonEmpty (a -> a) -> a -> a
    linkUp = Prelude.foldr1 (.)

    -- | Boilerplate transformation between our service structures and Servant's 'Handler' monad:
    toHandler :: AppEnv -> WebApp a -> Srv.Handler a
    toHandler e =
      Handler . withExceptT asHttpError . flip runReaderT e . runApp

    fakeSessionValidation :: BasicAuthData -> IO (AuthResult SessionContext)
    fakeSessionValidation _ =
      pure $ Authenticated $ SessionContext 1

Finally here's the main function for executing the service: `listen`. It's using the [Continuations](https://hackage.haskell.org/package/mtl-2.3.1/docs/Control-Monad-Cont.html) package, which is Haskell ecosystem main formalization for for high-level complex processing environments where there will be multiple processes, interruptions, initialization and cleanup requirements, and etc.  The function is most importantly a call to `runContT` which in turn will invoke:

    webHandling <- runAPI rtOpts

that sets up all the servicing logic for execution, and then

    runSettings settings webHandling

that will create the WAI HTTP listener and starts Servant main request processing logic.

Additionally it defines what happens before, after, how to gracefully shutdown the process, etc, which in this demo is all mock logic.

In [242]:
listen :: Ropt.RunOptions -> IO ()
listen rtOpts = do
  let
    fakeContT = ContT $ bracket (fakeContFct "dummy.") fakeEndFct
  runContT fakeContT finalAction
  where
  finalAction dummy = do
    let shutdownCallback = putStrLn "@[finalAction] empty termination callback."
        settings = setupWai rtOpts.webServer.port shutdownCallback
    webHandling <- runAPI rtOpts
    Wr.runSettings settings webHandling

  fakeContFct :: [a] -> IO Int
  fakeContFct l = return (Prelude.length l)

  fakeEndFct :: Int -> IO ()
  fakeEndFct aNum = pure ()

Like for the WAI/Warp example before, running the web service is done in a sub-thread with a call to the `forkIO` function.  The next line will spawn a full blown web server that is a realistic template for advanced and modern web app implementation:

In [244]:
tid <- Cc.forkIO (listen defaultRun)
:! date

Thu Dec  7 06:39:56 AM +04 2023

In [243]:
Cc.killThread tid
:! date

Thu Dec  7 06:39:54 AM +04 2023

<h1>References</h1>

- [GitHub Top 100](https://github.com/EvanLi/Github-Ranking)
- [Youtube Embedding](https://codepen.io/orvilleChomer/pen/mYBJzE)
